#<img alt="Creative Commons License" style="border-width:0" src="https://s3-us-west-2.amazonaws.com/webresources-savingforcollege/images/school_logos/san-jose-state-university.png" /></a> <span style="font-family:castellar; color:#2607d9";>Team Spartan Project: BUS4-118D </span>
### Ali Alzghari - Anthony Jason Villaroman - Duy Doan - Eiman Chaudhry - James Rudeen

The purpose of this notebook is to memorialize how we address a buisness need hypotheitcally proposed by <a href="https://www.yelp.com" target="_blank">Yelp</a>. Using Yelp's sample data, we demonstrate our methdods for accessing the data, profiling and transforming those data, and ultimately publishing meaningful output in a graphical way. 

The business need this project addresses rises from the following questions: 
<ul>
<li>Users can vote on the reviews by other users as "funny," "useful," or "cool." Do reviews of non-chain restaurants get more votes or more often get voted on than reviews of chain restaurants?</li>
  <ul>
  <li><i>i.e. does a restaurant's affiliation with others relate to prospects' offering of votes to reviews of that restaurant, and is there any pattern or consistency in that relationship accross metro areas?</i>
  </ul>
<li>Are there different patterns for "funny," "useful," or "cool" votes?</li>
  <ul>
  <li><i>Each vote option is equally subjective, but does any stand out in number over the others, and is that trend consistent throughout metro areas?</i>
  </ul>
<li>Does this vary geographically? In other words, are there just more users voting in a big city?</li>
  <ul>
  <li><i>In order to assess any derived implications the data present regarding the first two questions, we must determine if any such indicators are driven merely by the size of the metro area in view.</i>
  </ul>
</ul>

<p style="color:blue;"><b>Working note:</b> When attaching a cluster, name it "Spartan" so an ODBC connection can consistently be referenced among individual instances of Tableau.</p>

#Part 1:Accessing the Data

Through exercise of the CoNVO (context, need, vision, output) methodology, we have determined a vision of what our output will be. From that practice, we have reverse-engineered the requisite data to achieve that. 
These data will come from the following sources: 
<ul>
<li>Yelp's sample data</li>
<li>Supplimental data related to populations</li>
<li>Chain and non-chain restarurant distinctions</li>
</ul>

##Part 1A: Loading the Yelp Data

Yelp annually publishes a set of <a href="https://www.yelp.com/dataset" target="_blank">sample data</a> that include reviews from its users within ten major metroplitan areas in North America. That dataset includes the following components:
<ul>
  <li>business.bz2 - a listing of individual metro area businesses and their attributes</li>
  <li>checkin.bz2 - a record of reviewers' interaction with metro area businesses</li>
  <li>review.bz2 - all reviews of metro area businesses and types of votes they received</li>
  <li>tip.bz2 - record of tips offered by reviewers to metro area businesses</li>
  <li>user.bz2 - profile informaton related to Yelp users</li>
</ul>

Of these, only the <i>business.bz2</i> and <i>review.bz2</i>` data are required for our purpose. However, due to bandwidth variations and decentralization of users, the full suite of these data have been pulled from a zipped data source at Amazon's S3 storage service. 

We have removed the code that perform these actions to tidy this notebook. However, in the following cell, we examine the contents of the `/yelp` directory, and verify their number and respective sizes match those downloaded.

In [0]:
# List all files within the /yelp directory
dbutils.fs.ls("/yelp")

Out[1]: [FileInfo(path='dbfs:/yelp/business.bz2', name='business.bz2', size=15732814),
 FileInfo(path='dbfs:/yelp/checkin.bz2', name='checkin.bz2', size=85040781),
 FileInfo(path='dbfs:/yelp/covid.bz2', name='covid.bz2', size=5457206),
 FileInfo(path='dbfs:/yelp/review.bz2', name='review.bz2', size=1907673225),
 FileInfo(path='dbfs:/yelp/tip.bz2', name='tip.bz2', size=78821693),
 FileInfo(path='dbfs:/yelp/user.bz2', name='user.bz2', size=1793176640)]

##Part 1B: Collecting Other Required Data

In addition to the Yelp data, we will need more from which we can accurately reference certain measure and categorization:
<ul>
  <li>population of the ten metro areas covered within the Yelp sample data</li>
  <li>a listing of restaruant banners that are known chains across North America</li>
</ul>

###Referencing Metro Populations

Because the data for population are so small and lack complexity, one small DataFrame is created to store the values of each metropolitan area, and the most recent estimated population from census'. Compiled and current lists exists at the <a href="https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1710013501" target="_blank">Canada Statistics</a> and <a href="https://www.census.gov/programs-surveys/metro-micro/data/tables.2019.html" target="_blank">US Census</a> websites.

In [0]:
# Establish a DataFrame current populations of each of the ten metro areas of interest
df_metroPopulation = spark.createDataFrame(
     [("CAN","Montreal",4221000), 
      ("CAN","Toronto",6472000), 
      ("CAN","Calgary",1547000),
      ("USA","Pittsburgh",1704000),
      ("USA","Charlotte",2054000),
      ("USA","Urbana-Champaign",250000),
      ("USA","Phoenix",4511000),
      ("USA","Las Vegas",2699000),
      ("USA","Madison",578000),
      ("USA","Cleveland",1763000)],
# Assign column names
["country","metro_area","population"])
# Create a temporary view from the DataFrame
df_metroPopulation.createOrReplaceTempView("Metro_Population")
# Display the resulting DataFrame
df_metroPopulation.show()

+-------+----------------+----------+
country| metro_area|population|
+-------+----------------+----------+
 CAN| Montreal| 4221000|
 CAN| Toronto| 6472000|
 CAN| Calgary| 1547000|
 USA| Pittsburgh| 1704000|
 USA| Charlotte| 2054000|
 USA|Urbana-Champaign| 250000|
 USA| Phoenix| 4511000|
 USA| Las Vegas| 2699000|
 USA| Madison| 578000|
 USA| Cleveland| 1763000|
+-------+----------------+----------+

###Importing Chain and Non-Chain Restaruant Banners
With no budget for polling subscription-based services that provide banner names of chain restaurants in North America, we have found cost-free, robust listings at <a href="https://en.wikipedia.org/wiki/List_of_restaurant_chains_in_the_United_States" target="_blank">Wikipedia (USA chains)</a> and <a href="https://en.wikipedia.org/wiki/List_of_Canadian_restaurant_chains" target="_blank">Wikipedia (CAN chains)</a>. From these, we compile a simple text file list of the combined (and distinct) afilliations. That file is imported into Databricks using the UI (drag and drop). 

In the following cell, we create a unique `/chain` directory to hold this list file (ChainRestaurants.txt), port it into that directory, and then verify its transport.

In [0]:
# Call the functional catalog required for importing
import os.path
from os import path
# Create a directory to hold the data 
dbutils.fs.mkdirs("/chains")
# Move the imported data into the /chains directory
if path.exists("dbfs:/FileStore/tables/ChainRestaurants.txt")==True:
  dbutils.fs.mv("dbfs:/FileStore/tables/ChainRestaurants.txt","dbfs:/chains")
# List the files within the /chains directory
dbutils.fs.ls("dbfs:/chains")

Out[3]: [FileInfo(path='dbfs:/chains/ChainRestaurants.txt', name='ChainRestaurants.txt', size=7892)]

#Part 2: Profiling and Transforming the Data

With our data accessed, we now work to structure, enrich, and cleanse them to use in the Tableau analytics tool for visual presentation.

##Part 2A: Structure the Reference Data

The data imported for reference must be structured into a usable way for downstream profiling and transformation.

###Create a DataFrame of the Chain Restaurant Names for Later Use

In the following cell, we read the ChainRestaurants.txt file and write its content into an initial `df_chainRestaurant` DataFrame draft. Next, we create a transformed DataFrame to produce a meaningful header for the singular field (`chain`). This will make downstream profiling easiser in referencing these data. Finally, we display the new DataFrame (`df_chainNames`) to inspect its quality.

In [0]:
# Load the chainRestaurant data
df_chainRestaurant = spark.read.option("header","false").option("inferSchema","true").csv("dbfs:/chains/*.txt")
# Modify the column name for easier reference and generate a new DataFrame called dfChainNames
df_chainName = df_chainRestaurant.toDF("chain")
# Display a count of the records
print("Record count:", df_chainRestaurant.count())
# Display the resulting DataFrame
df_chainName.show(truncate=False)

Record count: 464
+----------------------------------+
chain |
+----------------------------------+
241 Pizza |
A&W (Canada) |
A&W Restaurants |
America's Incredible Pizza Company|
Applebee's |
Arby's |
Arctic Circle Restaurants |
Argo Tea |
Arthur Treacher's |
Atlanta Bread Company |
Au Bon Pain |
Auntie Anne's |
Back Yard Burgers |
Bahama Breeze |
Baja Fresh |
Bakers Square |
Bar Louie |
Baskin-Robbins |
Baton Rouge (restaurant) |
BeaverTails/Queues de Castor |
+----------------------------------+
only showing top 20 rows

###Create a Temporary View of the Chain Restauraunt Names for Later Use

In the following cell, we create from the `df_chainName` DataFrame a temporary view named `Chain_Name`. In addition of creating this view for downstream use in profiling, we perform the import of the `pyspark.sql.functions` tools into the notebook's state. Using its regular expressions library, we eliminate non-alphanumeric characters from the chain name and set what remains to strictly upper-case. This will provide an ASCII value of the chain name for precision comparison in later `JOIN` functions. In less technical terms, we are establishing a way to compare apples to apples regarding restaurant names among disparate sets of data.

In [0]:
# Call the functional catalog required for importing
import pyspark.sql.functions as f
# Create a temporary view from the DataFrame
df_chainName.createOrReplaceTempView("Chain_Name")
df_chainName = spark.sql("""
SELECT UPPER(REGEXP_REPLACE(chain,"[^0-9a-zA-Z$]+", "")) AS chain
  FROM Chain_Name
""")
# Replace the temporary view from the DataFrame
df_chainName.createOrReplaceTempView("Chain_Name")
# Display the resulting DataFrame
df_chainName.show(truncate = False)

+------------------------------+
chain |
+------------------------------+
241PIZZA |
AWCANADA |
AWRESTAURANTS |
AMERICASINCREDIBLEPIZZACOMPANY|
APPLEBEES |
ARBYS |
ARCTICCIRCLERESTAURANTS |
ARGOTEA |
ARTHURTREACHERS |
ATLANTABREADCOMPANY |
AUBONPAIN |
AUNTIEANNES |
BACKYARDBURGERS |
BAHAMABREEZE |
BAJAFRESH |
BAKERSSQUARE |
BARLOUIE |
BASKINROBBINS |
BATONROUGERESTAURANT |
BEAVERTAILSQUEUESDECASTOR |
+------------------------------+
only showing top 20 rows

###Create a DataFrame of the Yelp Business Data for Later Use

Next, we read the <i>business.bz2</i> file, write it into a `df_business` DataFrame, and then display the new DataFrame to inspect its quality. During this load, we are taking opportunity to sequence the columns in a logical way for better reference.

In [0]:
# Load the business.bz2 data
df_businessData = spark.read.json('/yelp/business.bz2').select("business_id","name","address","city","state","postal_code","latitude","longitude","hours","categories","attributes","review_count","stars","is_open")
# Display a count of the records
print("Record count:",df_businessData.count())
# Display the resulting DataFrame
df_businessData.show(10, truncate=75, vertical=True)

Record count: 209393
-RECORD 0-----------------------------------------------------------------------------------
 business_id | f9NumwFMBDn751xgFiRbNA 
 name | The Range At Lake Norman 
 address | 10913 Bailey Rd 
 city | Cornelius 
 state | NC 
 postal_code | 28031 
 latitude | 35.4627242 
 longitude | -80.8526119 
 hours | [11:0-20:0, 10:0-18:0, 11:0-20:0, 13:0-18:0, 11:0-20:0, 11:0-20:0, 10:0-... 
 categories | Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping 
 attributes | [,,,,,,, True,, True, {'garage': False, 'street': False, 'validated': Fa... 
 review_count | 36 
 stars | 3.5 
 is_open | 1 
-RECORD 1-----------------------------------------------------------------------------------
 business_id | Yzvjg0SayhoZgCljUJRF9Q 
 name | Carlos Santo, NMD 
 address | 8880 E Via Linda, Ste 107 
 city | Scottsdale 
 state | AZ 
 postal_code | 85258 
 latitude | 33.5694041 
 longitude | -111.8902637 
 hours | null 
 categories | Health & Medical, Fitness & Instruction, Yoga, Active Life, Pilates 
 attributes | [,,,,,,,,,,, True,,,,,,,, True,,,,,,,,,,,,,,,,,,,] 
 review_count | 4 
 stars | 5.0 
 is_open | 1 
-RECORD 2-----------------------------------------------------------------------------------
 business_id | XNoUzKckATkOD1hP6vghZg 
 name | Felinus 
 address | 3554 Rue Notre-Dame O 
 city | Montreal 
 state | QC 
 postal_code | H4C 1P4 
 latitude | 45.479984 
 longitude | -73.58007 
 hours | null 
 categories | Pets, Pet Services, Pet Groomers 
 attributes | null 
 review_count | 5 
 stars | 5.0 
 is_open | 1 
-RECORD 3-----------------------------------------------------------------------------------
 business_id | 6OAZjbxqM5ol29BuHsil3w 
 name | Nevada House of Hose 
 address | 1015 Sharp Cir 
 city | North Las Vegas 
 state | NV 
 postal_code | 89030 
 latitude | 36.2197281 
 longitude | -115.1277255 
 hours | [7:0-16:0, 7:0-16:0,,, 7:0-16:0, 7:0-16:0, 7:0-16:0] 
 categories | Hardware Stores, Home Services, Building Supplies, Home & Garden, Shopping 
 attributes | [,,,,,,, True,, True, {'garage': False, 'street': False, 'validated': Fa... 
 review_count | 3 
 stars | 2.5 
 is_open | 0 
-RECORD 4-----------------------------------------------------------------------------------
 business_id | 51M2Kk903DFYI6gnB5I6SQ 
 name | USE MY GUY SERVICES LLC 
 address | 4827 E Downing Cir 
 city | Mesa 
 state | AZ 
 postal_code | 85205 
 latitude | 33.4280652 
 longitude | -111.7266485 
 hours | [9:0-16:0, 0:0-0:0,,, 9:0-16:0, 9:0-16:0, 9:0-16:0] 
 categories | Home Services, Plumbing, Electricians, Handyman, Contractors 
 attributes | [,,,,,,,,, True,, True,,,,,,,,,,,,,,,,,,,,,,,,,,,] 
 review_count | 26 
 stars | 4.5 
 is_open | 1 
-RECORD 5-----------------------------------------------------------------------------------
 business_id | cKyLV5oWZJ2NudWgqs8VZw 
 name | Oasis Auto Center - Gilbert 
 address | 1720 W Elliot Rd, Ste 105 
 city | Gilbert 
 state | AZ 
 postal_code | 85233 
 latitude | 33.3503993 
 longitude | -111.8271417 
 hours | [7:0-18:0, 7:0-18:0, 7:0-15:0,, 7:0-18:0, 7:0-18:0, 7:0-18:0] 
 categories | Auto Repair, Automotive, Oil Change Stations, Transmission Repair 
 attributes | [,,,,,,,,, True,,,,,,,,,,,,,,,,,,,,,,,,,,,,,] 
 review_count | 38 
 stars | 4.5 
 is_open | 1 
-RECORD 6-----------------------------------------------------------------------------------
 business_id | oiAlXZPIFm2nBCt0DHLu_Q 
 name | Green World Cleaners 
 address | 6870 S Rainbow Blvd, Ste 117 
 city | Las Vegas 
 state | NV 
 postal_code | 89118 
 latitude | 36.0639767 
 longitude | -115.241463 
 hours | [7:0-19:0, 7:0-19:0, 9:0-17:0,, 7:0-19:0, 7:0-19:0, 7:0-19:0] 
 categories | Dry Cleaning & Laundry, Local Services, Laundry Services 
 attributes | [,,,,,,, True,, True, {'garage': False, 'street': False, 'validated': Fa... 
 review_count | 81 
 stars | 3.5 
 is_open | 1 
-RECORD 7-----------------------------------------------------------------------------------
 business_id | ScYkbYNkDgCneBrD9vqhCQ 
 name | Junction Tire & Au

###Create a Temporary View of the Yelp Business Data for Later Use

From the `df_businessData` DataFrame, we now create a temporary view named `Business_Data` for downstream use in profiling.

In [0]:
# Create a temporary view of the Yelp business data
df_businessData.createOrReplaceTempView("Business_Data")
spark.sql("""
SELECT business_id,
       name,
       address,
       city,
       state,
       postal_code,
       latitude,
       longitude,
       hours,
       categories,
       attributes,
       review_count,
       stars,
       is_open       
  FROM Business_Data
""").show(10, truncate=75, vertical=True)

-RECORD 0-----------------------------------------------------------------------------------
 business_id | f9NumwFMBDn751xgFiRbNA 
 name | The Range At Lake Norman 
 address | 10913 Bailey Rd 
 city | Cornelius 
 state | NC 
 postal_code | 28031 
 latitude | 35.4627242 
 longitude | -80.8526119 
 hours | [11:0-20:0, 10:0-18:0, 11:0-20:0, 13:0-18:0, 11:0-20:0, 11:0-20:0, 10:0-... 
 categories | Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping 
 attributes | [,,,,,,, True,, True, {'garage': False, 'street': False, 'validated': Fa... 
 review_count | 36 
 stars | 3.5 
 is_open | 1 
-RECORD 1-----------------------------------------------------------------------------------
 business_id | Yzvjg0SayhoZgCljUJRF9Q 
 name | Carlos Santo, NMD 
 address | 8880 E Via Linda, Ste 107 
 city | Scottsdale 
 state | AZ 
 postal_code | 85258 
 latitude | 33.5694041 
 longitude | -111.8902637 
 hours | null 
 categories | Health & Medical, Fitness & Instruction, Yoga, Active Life, Pilates 
 attributes | [,,,,,,,,,,, True,,,,,,,, True,,,,,,,,,,,,,,,,,,,] 
 review_count | 4 
 stars | 5.0 
 is_open | 1 
-RECORD 2-----------------------------------------------------------------------------------
 business_id | XNoUzKckATkOD1hP6vghZg 
 name | Felinus 
 address | 3554 Rue Notre-Dame O 
 city | Montreal 
 state | QC 
 postal_code | H4C 1P4 
 latitude | 45.479984 
 longitude | -73.58007 
 hours | null 
 categories | Pets, Pet Services, Pet Groomers 
 attributes | null 
 review_count | 5 
 stars | 5.0 
 is_open | 1 
-RECORD 3-----------------------------------------------------------------------------------
 business_id | 6OAZjbxqM5ol29BuHsil3w 
 name | Nevada House of Hose 
 address | 1015 Sharp Cir 
 city | North Las Vegas 
 state | NV 
 postal_code | 89030 
 latitude | 36.2197281 
 longitude | -115.1277255 
 hours | [7:0-16:0, 7:0-16:0,,, 7:0-16:0, 7:0-16:0, 7:0-16:0] 
 categories | Hardware Stores, Home Services, Building Supplies, Home & Garden, Shopping 
 attributes | [,,,,,,, True,, True, {'garage': False, 'street': False, 'validated': Fa... 
 review_count | 3 
 stars | 2.5 
 is_open | 0 
-RECORD 4-----------------------------------------------------------------------------------
 business_id | 51M2Kk903DFYI6gnB5I6SQ 
 name | USE MY GUY SERVICES LLC 
 address | 4827 E Downing Cir 
 city | Mesa 
 state | AZ 
 postal_code | 85205 
 latitude | 33.4280652 
 longitude | -111.7266485 
 hours | [9:0-16:0, 0:0-0:0,,, 9:0-16:0, 9:0-16:0, 9:0-16:0] 
 categories | Home Services, Plumbing, Electricians, Handyman, Contractors 
 attributes | [,,,,,,,,, True,, True,,,,,,,,,,,,,,,,,,,,,,,,,,,] 
 review_count | 26 
 stars | 4.5 
 is_open | 1 
-RECORD 5-----------------------------------------------------------------------------------
 business_id | cKyLV5oWZJ2NudWgqs8VZw 
 name | Oasis Auto Center - Gilbert 
 address | 1720 W Elliot Rd, Ste 105 
 city | Gilbert 
 state | AZ 
 postal_code | 85233 
 latitude | 33.3503993 
 longitude | -111.8271417 
 hours | [7:0-18:0, 7:0-18:0, 7:0-15:0,, 7:0-18:0, 7:0-18:0, 7:0-18:0] 
 categories | Auto Repair, Automotive, Oil Change Stations, Transmission Repair 
 attributes | [,,,,,,,,, True,,,,,,,,,,,,,,,,,,,,,,,,,,,,,] 
 review_count | 38 
 stars | 4.5 
 is_open | 1 
-RECORD 6-----------------------------------------------------------------------------------
 business_id | oiAlXZPIFm2nBCt0DHLu_Q 
 name | Green World Cleaners 
 address | 6870 S Rainbow Blvd, Ste 117 
 city | Las Vegas 
 state | NV 
 postal_code | 89118 
 latitude | 36.0639767 
 longitude | -115.241463 
 hours | [7:0-19:0, 7:0-19:0, 9:0-17:0,, 7:0-19:0, 7:0-19:0, 7:0-19:0] 
 categories | Dry Cleaning & Laundry, Local Services, Laundry Services 
 attributes | [,,,,,,, True,, True, {'garage': False, 'street': False, 'validated': Fa... 
 review_count | 81 
 stars | 3.5 
 is_open | 1 
-RECORD 7-----------------------------------------------------------------------------------
 business_id | ScYkbYNkDgCneBrD9vqhCQ 
 name | Junction Tire & Auto Service 
 address 

###Create a DataFrame of the Yelp Review Data for Later Use

In the following cell, we read the <i>review.bz2</i> file and write it into a `df_reviewData` DataFrame and then display the new DataFrame to inspect its quality. During this load, we are taking opportunity to refine the structure by pulling fields that are relevent to our question and sequence the columns in a logical way for better reference.

In [0]:
# Load the review.bz2 data
if ( spark.catalog._jcatalog.tableExists("reviews_without_text_table") ):
  print("Loading review data from table ...")
  df_reviewData = spark.read.format('parquet').table("reviews_without_text_table").cache()
  print("loaded review data from table")
else:
  print("The Building Review and User Tables notebook has not been run, loading review data from files ...")
  df_reviewData = spark.read.json("/yelp/review.bz2").select("business_id","user_id","cool","funny","useful","date","stars").cache()
  # Does not already exist as a table, so write it out
  df_reviewData.write.mode("overwrite").format('parquet').saveAsTable("reviews_without_text_table")
  print("loaded review data from file")
# Display a count of the records
print("Record count:",df_reviewData.count())
# Display the resulting DataFrame
df_reviewData.show()

Loading review data from table ...
loaded review data from table
Record count: 8021122
+--------------------+--------------------+----+-----+------+-------------------+-----+
 business_id| user_id|cool|funny|useful| date|stars|
+--------------------+--------------------+----+-----+------+-------------------+-----+
3chd-CjwQz7Pap_MQ...|c4Y3EX5aXYixkyVD6...| 2| 1| 1|2019-09-04 19:16:55| 4.0|
XV45a3fz1d7SRoash...|fHTAUYCd5HuboQp6H...| 1| 1| 13|2016-03-01 21:46:00| 3.0|
BxcLVgA6BndKDJol0...|ecXDhY91U-ORKk_U7...| 0| 0| 0|2019-11-18 07:13:06| 3.0|
nSPh4_BKX3Fch0TOa...|ZKpETYqLymDFL76hw...| 0| 0| 3|2019-08-15 16:30:53| 1.0|
KrcmoxVBX9o3OxFog...|zcp389V72h0DKl69X...| 4| 3| 6|2019-10-24 20:37:02| 3.0|
ewn93D_Q6bnVosJtw...|xbksPRSmtsmDNiR1V...| 0| 0| 0|2019-11-19 15:33:33| 2.0|
fKjeOlJTQ1W0cSDRY...|dG-J0IG122Nwqp0Co...| 0| 0| 1|2019-10-21 00:15:38| 1.0|
b7xistZfcbSO6_0Mg...|O4ADr-eEYkP1OKQUx...| 0| 0| 13|2017-12-22 07:26:39| 1.0|
6pM2YxQH8dQYs07_C...|LeSXKUJeFvX2aSvfP...| 0| 0| 2|2019-04-15 14:20:28| 5.0|
iVjeR9z1xVt2ZpxVU...|_fX--5YqPXMB4b7q2...| 0| 0| 0|2019-11-16 22:29:28| 5.0|
t0edCFMUYmFSN_eXT...|HhfNSpdHGSGiuyi1a...| 0| 0| 4|2019-08-13 03:02:45| 4.0|
TumOl-iIDkZFRM1k9...|YKiRYAlL7SEV5P29p...| 0| 0| 3|2019-03-15 20:26:47| 5.0|
ndjiWXoH1pPMwEBhR...|Zt_z6uOk0fPXjsFvC...| 2| 0| 2|2019-11-19 14:47:04| 5.0|
z8lagvcw1Y4Pz4dVN...|y0HCy6GpDFnsQXhCG...| 0| 0| 0|2019-11-24 18:17:55| 5.0|
0OZckGHdpbhMMy8ex...|SB7Ow6Y52kNcHdecd...| 0| 0| 19|2018-04-06 03:23:10| 5.0|
d10IxZPirVJlOSpdR...|0SrL97xFh-mVMOxUF...| 0| 1| 3|2017-03-10 03:47:23| 1.0|
J3EW9_IQQqkp4nN3q...|yZ3Zm1fBfqlBWhFik...| 0| 0| 0|2015-04-06 00:50:59| 4.0|
E9HkUYYO_-5vKjvBw...|hRzmJCPmV1_ggM5jS...| 0| 0| 1|2019-10-26 04:00:14| 5.0|
lNBhwf_3c0QBqBlsY...|HS3y3GltOH_pJ37-E...| 0| 0| 0|2019-11-27 14:34:34| 4.0|
Kp4i7on7rxFRkWceK...|R-qaBztGNZoxEbKet...| 0| 0| 0|2015-04-27 07:48:39| 5.0|
+--------------------+--------------------+----+-----+------+-------------------+-----+
only showing top 20 rows

###Create a Temporary View of the Yelp Review Data for Later Use

Now, we create from the `df_reviewData` DataFrame a temporary view named `Review_Data` for downstream use in profiling.

In [0]:
# Create a temporary view of the Yelp review data
df_reviewData.createOrReplaceTempView("Review_Data")
spark.sql("""
SELECT business_id,
       user_id,
       cool,
       funny,
       useful,
       date,
       stars  
  FROM Review_Data
""").show()

+--------------------+--------------------+----+-----+------+-------------------+-----+
 business_id| user_id|cool|funny|useful| date|stars|
+--------------------+--------------------+----+-----+------+-------------------+-----+
3chd-CjwQz7Pap_MQ...|c4Y3EX5aXYixkyVD6...| 2| 1| 1|2019-09-04 19:16:55| 4.0|
XV45a3fz1d7SRoash...|fHTAUYCd5HuboQp6H...| 1| 1| 13|2016-03-01 21:46:00| 3.0|
BxcLVgA6BndKDJol0...|ecXDhY91U-ORKk_U7...| 0| 0| 0|2019-11-18 07:13:06| 3.0|
nSPh4_BKX3Fch0TOa...|ZKpETYqLymDFL76hw...| 0| 0| 3|2019-08-15 16:30:53| 1.0|
KrcmoxVBX9o3OxFog...|zcp389V72h0DKl69X...| 4| 3| 6|2019-10-24 20:37:02| 3.0|
ewn93D_Q6bnVosJtw...|xbksPRSmtsmDNiR1V...| 0| 0| 0|2019-11-19 15:33:33| 2.0|
fKjeOlJTQ1W0cSDRY...|dG-J0IG122Nwqp0Co...| 0| 0| 1|2019-10-21 00:15:38| 1.0|
b7xistZfcbSO6_0Mg...|O4ADr-eEYkP1OKQUx...| 0| 0| 13|2017-12-22 07:26:39| 1.0|
6pM2YxQH8dQYs07_C...|LeSXKUJeFvX2aSvfP...| 0| 0| 2|2019-04-15 14:20:28| 5.0|
iVjeR9z1xVt2ZpxVU...|_fX--5YqPXMB4b7q2...| 0| 0| 0|2019-11-16 22:29:28| 5.0|
t0edCFMUYmFSN_eXT...|HhfNSpdHGSGiuyi1a...| 0| 0| 4|2019-08-13 03:02:45| 4.0|
TumOl-iIDkZFRM1k9...|YKiRYAlL7SEV5P29p...| 0| 0| 3|2019-03-15 20:26:47| 5.0|
ndjiWXoH1pPMwEBhR...|Zt_z6uOk0fPXjsFvC...| 2| 0| 2|2019-11-19 14:47:04| 5.0|
z8lagvcw1Y4Pz4dVN...|y0HCy6GpDFnsQXhCG...| 0| 0| 0|2019-11-24 18:17:55| 5.0|
0OZckGHdpbhMMy8ex...|SB7Ow6Y52kNcHdecd...| 0| 0| 19|2018-04-06 03:23:10| 5.0|
d10IxZPirVJlOSpdR...|0SrL97xFh-mVMOxUF...| 0| 1| 3|2017-03-10 03:47:23| 1.0|
J3EW9_IQQqkp4nN3q...|yZ3Zm1fBfqlBWhFik...| 0| 0| 0|2015-04-06 00:50:59| 4.0|
E9HkUYYO_-5vKjvBw...|hRzmJCPmV1_ggM5jS...| 0| 0| 1|2019-10-26 04:00:14| 5.0|
lNBhwf_3c0QBqBlsY...|HS3y3GltOH_pJ37-E...| 0| 0| 0|2019-11-27 14:34:34| 4.0|
Kp4i7on7rxFRkWceK...|R-qaBztGNZoxEbKet...| 0| 0| 0|2015-04-27 07:48:39| 5.0|
+--------------------+--------------------+----+-----+------+-------------------+-----+
only showing top 20 rows

##Part 2B: Data Profiling

Here, we perform steps to focus, enhance, and clean the data to improve its quality for output. Also, we validate the distribution of values for given fields across multiple records. We examine the shape and extent of the distribution of values for realistic expectation and consistency.

###Enhance the Reviews to Subtotal by Business

In the following cells, we calculate a novel column for inclusion to the `df_reviewData` DataFrame to display the total number of reviews for the `business_id` in each row. Having this total will provide opportunity for later, weighted calculations related to votes and reviews.

In [0]:
# Calculate a subtotal of reviews for each business and display those in a novel field
df_reviewData = spark.sql("""
    SELECT A.business_id,
           user_id,
           cool,
           funny,
           useful,
           date,
           stars,
           subtotal_review
      FROM Review_Data AS A
INNER JOIN
           (  SELECT business_id, 
                     COUNT(user_id) AS subtotal_review
                FROM Review_Data
            GROUP BY business_id) AS B
        ON A.business_id == B.business_id
  ORDER BY A.business_id
""")
# Display a count of the records
print("Record count:",df_reviewData.count())
# Display the resulting DataFrame
df_reviewData.show()
# Create a temporary view of the Yelp review data
df_reviewData.createOrReplaceTempView("Review_Data")

Record count: 8021122
+--------------------+--------------------+----+-----+------+-------------------+-----+---------------+
 business_id| user_id|cool|funny|useful| date|stars|subtotal_review|
+--------------------+--------------------+----+-----+------+-------------------+-----+---------------+
--1UhMGODdWsrMast...|NqpKiaRsGfuU2voV5...| 2| 0| 5|2016-06-04 20:57:29| 1.0| 29|
--1UhMGODdWsrMast...|t4cYW73lVcBb-1R_W...| 0| 0| 0|2016-07-25 23:31:01| 4.0| 29|
--1UhMGODdWsrMast...|P3V5p1Wt3znW2-4Yt...| 0| 0| 0|2017-12-23 00:49:01| 5.0| 29|
--1UhMGODdWsrMast...|brd33PD_6nqK_VVnO...| 0| 0| 2|2016-04-21 18:21:32| 4.0| 29|
--1UhMGODdWsrMast...|hqk4eugYhjmhM-3S2...| 1| 0| 0|2016-12-06 20:04:46| 5.0| 29|
--1UhMGODdWsrMast...|P3V5p1Wt3znW2-4Yt...| 0| 0| 0|2017-06-17 20:06:11| 5.0| 29|
--1UhMGODdWsrMast...|q0ebW-bQD9UOEQmpw...| 0| 0| 0|2018-12-07 03:54:55| 5.0| 29|
--1UhMGODdWsrMast...|m-p-7WuB85UjsLDax...| 1| 0| 1|2018-02-25 17:47:12| 5.0| 29|
--1UhMGODdWsrMast...|NoQCmYKyMPs4D01Wa...| 0| 0| 0|2017-08-09 18:12:06| 4.0| 29|
--1UhMGODdWsrMast...|FYhU1fKQ7n11WQ7gc...| 0| 0| 0|2018-01-11 19:55:31| 5.0| 29|
--1UhMGODdWsrMast...|A4bpHuvzaQt9-XAg8...| 0| 0| 0|2016-06-13 19:01:34| 3.0| 29|
--1UhMGODdWsrMast...|TzU30D-CjtPP3Xumg...| 1| 0| 0|2017-05-09 00:32:24| 4.0| 29|
--1UhMGODdWsrMast...|wO6innof5WG-1JeRq...| 0| 0| 0|2019-09-24 01:37:46| 3.0| 29|
--1UhMGODdWsrMast...|atyCaAjUYatIFDOGK...| 1| 0| 0|2017-05-10 17:34:06| 5.0| 29|
--1UhMGODdWsrMast...|NBQnPd9dhwRkCvZRg...| 0| 0| 1|2019-11-27 21:56:15| 5.0| 29|
--1UhMGODdWsrMast...|BgTWMo2qRrXINPiM3...| 1| 0| 1|2016-11-14 20:09:54| 4.0| 29|
--1UhMGODdWsrMast...|ncf_USO7wFU-B5bJj...| 0| 0| 0|2017-08-27 03:06:39| 4.0| 29|
--1UhMGODdWsrMast...|ZS_qdLMh1XyXKGTrp...| 2| 1| 2|2017-05-01 02:36:44| 5.0| 29|
--1UhMGODdWsrMast...|brd33PD_6nqK_VVnO...| 1| 0| 0|2016-09-07 15:18:25| 5.0| 29|
--1UhMGODdWsrMast...|ZgAzKwganIXImRAMc...| 1| 0| 0|2017-05-09 18:57:16| 4.0| 29|
+--------------------+--------------------+----+-----+------+-------------------+-----+---------------+
only showing top 20 rows

###Filter the Businesses to Only Restaurants

The focus of our questions is to restaurants. In the following cells, we distill the `df_businessData` DataFrame to only those businesses and preserve that record set in a unique DataFrame named `df_restaurantData`. We begin this process by transforming the `categories` field to an array by which we can filter by business type.

In [0]:
#Split the categories into an array
df_categories = spark.sql("""
SELECT business_id, 
       name,
       city,
       state,
       postal_code,
       latitude,
       longitude,
       attributes, 
       SPLIT(categories, '\\\s*,\\\s*') AS categories
  FROM Business_Data
""")
# Display a count of the records
print("Record count:", df_categories.count())
# Display the resulting DataFrame
df_categories.show(10, truncate = 75, vertical = True)
# Create a temporary view of restaurants
df_categories.createOrReplaceTempView("Business_Data_1")

Record count: 209393
-RECORD 0----------------------------------------------------------------------------------
 business_id | f9NumwFMBDn751xgFiRbNA 
 name | The Range At Lake Norman 
 city | Cornelius 
 state | NC 
 postal_code | 28031 
 latitude | 35.4627242 
 longitude | -80.8526119 
 attributes | [,,,,,,, True,, True, {'garage': False, 'street': False, 'validated': Fa... 
 categories | [Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping] 
-RECORD 1----------------------------------------------------------------------------------
 business_id | Yzvjg0SayhoZgCljUJRF9Q 
 name | Carlos Santo, NMD 
 city | Scottsdale 
 state | AZ 
 postal_code | 85258 
 latitude | 33.5694041 
 longitude | -111.8902637 
 attributes | [,,,,,,,,,,, True,,,,,,,, True,,,,,,,,,,,,,,,,,,,] 
 categories | [Health & Medical, Fitness & Instruction, Yoga, Active Life, Pilates] 
-RECORD 2----------------------------------------------------------------------------------
 business_id | XNoUzKckATkOD1hP6vghZg 
 name | Felinus 
 city | Montreal 
 state | QC 
 postal_code | H4C 1P4 
 latitude | 45.479984 
 longitude | -73.58007 
 attributes | null 
 categories | [Pets, Pet Services, Pet Groomers] 
-RECORD 3----------------------------------------------------------------------------------
 business_id | 6OAZjbxqM5ol29BuHsil3w 
 name | Nevada House of Hose 
 city | North Las Vegas 
 state | NV 
 postal_code | 89030 
 latitude | 36.2197281 
 longitude | -115.1277255 
 attributes | [,,,,,,, True,, True, {'garage': False, 'street': False, 'validated': Fa... 
 categories | [Hardware Stores, Home Services, Building Supplies, Home & Garden, Shopp... 
-RECORD 4----------------------------------------------------------------------------------
 business_id | 51M2Kk903DFYI6gnB5I6SQ 
 name | USE MY GUY SERVICES LLC 
 city | Mesa 
 state | AZ 
 postal_code | 85205 
 latitude | 33.4280652 
 longitude | -111.7266485 
 attributes | [,,,,,,,,, True,, True,,,,,,,,,,,,,,,,,,,,,,,,,,,] 
 categories | [Home Services, Plumbing, Electricians, Handyman, Contractors] 
-RECORD 5----------------------------------------------------------------------------------
 business_id | cKyLV5oWZJ2NudWgqs8VZw 
 name | Oasis Auto Center - Gilbert 
 city | Gilbert 
 state | AZ 
 postal_code | 85233 
 latitude | 33.3503993 
 longitude | -111.8271417 
 attributes | [,,,,,,,,, True,,,,,,,,,,,,,,,,,,,,,,,,,,,,,] 
 categories | [Auto Repair, Automotive, Oil Change Stations, Transmission Repair] 
-RECORD 6----------------------------------------------------------------------------------
 business_id | oiAlXZPIFm2nBCt0DHLu_Q 
 name | Green World Cleaners 
 city | Las Vegas 
 state | NV 
 postal_code | 89118 
 latitude | 36.0639767 
 longitude | -115.241463 
 attributes | [,,,,,,, True,, True, {'garage': False, 'street': False, 'validated': Fa... 
 categories | [Dry Cleaning & Laundry, Local Services, Laundry Services] 
-RECORD 7----------------------------------------------------------------------------------
 business_id | ScYkbYNkDgCneBrD9vqhCQ 
 name | Junction Tire & Auto Service 
 city | Mesa 
 state | AZ 
 postal_code | 85209 
 latitude | 33.3938847 
 longitude | -111.6822257 
 attributes | [,,,,,,,,, True,, False,,,,,,,,,,,,,,,,,,,,,,,,,,,] 
 categories | [Auto Repair, Oil Change Stations, Automotive, Tires] 
-RECORD 8----------------------------------------------------------------------------------
 business_id | pQeaRpvuhoEqudo3uymHIQ 
 name | The Empanadas House 
 city | Champaign 
 state | IL 
 postal_code | 61820 
 latitude | 40.1104457 
 longitude | -88.2330726 
 attributes | [,, 'none',,,,, False,, True, None, False,,,,,,,, False,,,, True,, u'qui... 
 categories | [Ethnic Food, Food Trucks, Specialty Food, Imported Food, Argentine, Foo... 
-RECORD 9----------------------------------------------------------------------------------
 business_id | EosRKXIGeSWFYWwpkbhNnA 
 name | Xtreme Couture 
 city | Toronto 
 state | ON 
 postal_code | M8Z 5G3 
 latitude | 43.6245394916 
 longitude | -79.529

Now, we extract only businesses that are categorized as restaurants. At the same time, we again utilize regular expressions to add a version of the business name in a way that we can effectivley compare names accross disparate data sets (`name_formatted`).

In [0]:
# Extract only businesses that are categorized as restaurants
# We utilize regular expressions to transform the name field - providing a method to compare keys' ASCII values in join clauses of downstream queries
df_restaurantData = spark.sql("""
SELECT business_id, 
       TRIM(name) AS name,
       UPPER(REGEXP_REPLACE(name,"[^0-9a-zA-Z$]+", "")) AS name_formatted,
       TRIM(city) AS city,
       TRIM(state) AS state,
       postal_code,
       latitude,
       longitude, 
       attributes
  FROM Business_Data_1
 WHERE ARRAY_CONTAINS(categories,"Restaurants")
""")
# Display a count of the records
print("Record count:", df_restaurantData.count())
# Display the resulting DataFrame
df_restaurantData.show(10, truncate = 75, vertical=True)
# Create a temporary view of restaurants
df_restaurantData.createOrReplaceTempView("Restaurant_Data")

Record count: 63944
-RECORD 0-------------------------------------------------------------------------------------
 business_id | pQeaRpvuhoEqudo3uymHIQ 
 name | The Empanadas House 
 name_formatted | THEEMPANADASHOUSE 
 city | Champaign 
 state | IL 
 postal_code | 61820 
 latitude | 40.1104457 
 longitude | -88.2330726 
 attributes | [,, 'none',,,,, False,, True, None, False,,,,,,,, False,,,, True,, u'qui... 
-RECORD 1-------------------------------------------------------------------------------------
 business_id | CsLQLiRoafpJPJSkNX2h5Q 
 name | Middle East Deli 
 name_formatted | MIDDLEEASTDELI 
 city | Charlotte 
 state | NC 
 postal_code | 28205 
 latitude | 35.194894 
 longitude | -80.767442 
 attributes | [,,,,,,,,, True, {'garage': False, 'street': False, 'validated': False, ... 
-RECORD 2-------------------------------------------------------------------------------------
 business_id | eBEfgOPG7pvFhb2wcG9I7w 
 name | Philthy Phillys 
 name_formatted | PHILTHYPHILLYS 
 city | Aurora 
 state | ON 
 postal_code | L4G 7J1 
 latitude | 44.0109618 
 longitude | -79.448677 
 attributes | [,, u'none',,,,, True,,, {'garage': False, 'street': False, 'validated':... 
-RECORD 3-------------------------------------------------------------------------------------
 business_id | lu7vtrp_bE9PnxWfA8g4Pg 
 name | Banzai Sushi 
 name_formatted | BANZAISUSHI 
 city | Thornhill 
 state | ON 
 postal_code | L3T 5W4 
 latitude | 43.8204923 
 longitude | -79.3984661 
 attributes | [,, u'none',,,,,,,,,,,,,,,,, True,,,,,,,,,,, True, False,, False,, True,,,] 
-RECORD 4-------------------------------------------------------------------------------------
 business_id | 9sRGfSVEfLhN_km60YruTA 
 name | Apadana Restaurant 
 name_formatted | APADANARESTAURANT 
 city | Richmond Hill 
 state | ON 
 postal_code | L4E 1A5 
 latitude | 43.9470107964 
 longitude | -79.454861645 
 attributes | [,, u'full_bar', {'touristy': False, 'hipster': False, 'romantic': False... 
-RECORD 5-------------------------------------------------------------------------------------
 business_id | vjTVxnsQEZ34XjYNS-XUpA 
 name | Wetzel's Pretzels 
 name_formatted | WETZELSPRETZELS 
 city | Phoenix 
 state | AZ 
 postal_code | 85032 
 latitude | 33.602822 
 longitude | -111.983533 
 attributes | [,, u'none', {'romantic': False, 'intimate': False, 'touristy': False, '... 
-RECORD 6-------------------------------------------------------------------------------------
 business_id | fnZrZlqW1Z8iWgTVDfv_MA 
 name | Carl's Jr 
 name_formatted | CARLSJR 
 city | Las Vegas 
 state | NV 
 postal_code | 89147 
 latitude | 36.0997379 
 longitude | -115.3015683 
 attributes | [,, u'none', {'romantic': False, 'intimate': False, 'classy': False, 'hi... 
-RECORD 7-------------------------------------------------------------------------------------
 business_id | rVBPQdeayMYht4Uv_FOLHg 
 name | Gourmet Burger Company 
 name_formatted | GOURMETBURGERCOMPANY 
 city | Toronto 
 state | ON 
 postal_code | M8Z 5G9 
 latitude | 43.6332914 
 longitude | -79.5317683 
 attributes | [,, u'none', {'romantic': False, 'intimate': False, 'classy': False, 'hi... 
-RECORD 8-------------------------------------------------------------------------------------
 business_id | 98hyK2QEUeI8v2y0AghfZA 
 name | Pho Lee's Vietnamese Restaurant 
 name_formatted | PHOLEESVIETNAMESERESTAURANT 
 city | Cleveland 
 state | OH 
 postal_code | 44114 
 latitude | 41.5121553 
 longitude | -81.663332 
 attributes | [,, u'none', {'touristy': False, 'hipster': False, 'romantic': False, 'd... 
-RECORD 9-------------------------------------------------------------------------------------
 business_id | fhNf_sg-XzZ3e7HEVGuOZg 
 name | Meat Chix And Wieners 
 name_formatted | MEATCHIXANDWIENERS 
 city | Las Vegas 
 state | NV 
 postal_code | 89118 
 latitude | 36.071196 
 longitude | -115.207463 
 attributes | [,, u'none', {'romantic': False, 'intimate': False, 'touristy': False, '... 
only showing top 10 rows

Before proceeding, we want to check for businesses among the data which list no value in the `category` field. If such businesses are of significant quantity, that it will distort the data since a portion of them could, in fact, be restaurants - orphaned from our analysis. 

To check this, we count businesses that are uncategorized, those categorized as restaurants, and then non-restaurant businesses. Displaying these counts graphically tells us the number of uncategorized entities in the business data (524) represent a fraction of one percent of all businesses. Therefore, information we derive from even the subset of restaurants (31% of the businesses) will be nebligibly skewed by businesses lacking any categorization.

In [0]:
# Determine the percentage of businesses that are uncategorized, restaurants, or another category
df_category = spark.sql("""
SELECT "uncategorized" AS category,
       COUNT(business_id) AS business_count
  FROM Business_Data
 WHERE categories IS NULL
 UNION
SELECT "restaurant" AS category,
       COUNT(business_id) AS business_count
  FROM Business_Data_1
 WHERE ARRAY_CONTAINS(categories,"Restaurants")
 UNION
SELECT "other_businesses" AS category,
       COUNT(business_id) AS business_count
  FROM Business_Data_1
  WHERE NOT ARRAY_CONTAINS(categories,"Restaurants")
""")
# Show the count results in a comparative way
df_category.display()

category,business_count
uncategorized,524
restaurant,63944
other_businesses,144925


###Determine the Definition of a Chain Restaurant

At this point, we would like to know which restaurants are members of a chain and which are non-affiliated. To do so, we first generate a listing of restaurant trade names and the associated count of those from our `Restaurant_Data` view.

In [0]:
# List restaurant names and total outlets under that banner
df_nameCount = spark.sql("""
  SELECT name_formatted, 
         COUNT(business_id) AS name_count
    FROM Restaurant_Data
GROUP BY name_formatted 
  HAVING COUNT(business_id) > 1
ORDER BY name_count DESC
""")
# Display the resulting DataFrame
df_nameCount.show(truncate = False)
# Create a temporary view of restaurant name counts
df_nameCount.createOrReplaceTempView("Name_Count")

+-----------------------+----------+
name_formatted |name_count|
+-----------------------+----------+
MCDONALDS |870 |
SUBWAYRESTAURANTS |613 |
BURGERKING |337 |
TACOBELL |335 |
PIZZAHUT |330 |
WENDYS |325 |
TIMHORTONS |263 |
SUBWAY |244 |
DOMINOSPIZZA |238 |
KFC |214 |
STARBUCKS |208 |
CHIPOTLEMEXICANGRILL |191 |
JACKINTHEBOX |188 |
JIMMYJOHNS |165 |
PANERABREAD |165 |
POPEYESLOUISIANAKITCHEN|161 |
PANDAEXPRESS |157 |
PAPAJOHNSPIZZA |156 |
ARBYS |148 |
LITTLECAESARSPIZZA |133 |
+-----------------------+----------+
only showing top 20 rows

Our challenge at this point is to form a logical determination of the count of demarcation between chain and non-chain restaurants. This point should be obvious when the data are viewed in a histogram. Seeing the jumps among counts however, we understand that portends a long-tail histographic representation. To squelch that noise, we establish a new DataFrame where the `name_count` value is capped at 20.

In [0]:
# Establish a dataframe that sets an upper bound to the count of affiliated restaurants among a single banner
df_cappedNameCount = spark.sql("""
  SELECT IF(name_count > 20, 20, name_count) AS capped_count
    FROM Name_Count
ORDER BY capped_count
""")

When graphing the result, we behold that a significant separation occurs between those restaurants with two locations and those with three or more. Therefore, in the case of these data, our definition of a chain restaurant will be: 
<i>A collection of three or more restaurants operating under a shared trade name.</i>

In [0]:
display(df_cappedNameCount)

capped_count
2
2
2
2
2
2
2
2
2
2


###Identify Chains not Already Accounted

With a non-abstract determiniation of what a chain restaurant is, we are now curious to know if the published list of chains from our Wikipedia sources circumscribe those in the Yelp data. In the cells that follow, we identify restaurants with multiple outlets yet are not in the known chain name data. These exceptions will be considered for inclusion as chain restaurants. We start by isolating restaurants that share the same name in at least three instances.

In [0]:
# List restaurant names that meet our definition of a chain
df_nameCount = spark.sql("""
  SELECT name_formatted, 
         COUNT(business_id) AS name_count
    FROM Restaurant_Data
   GROUP BY name_formatted 
  HAVING COUNT(name_formatted) >= 3
ORDER BY name_count DESC
""")
# Display the resulting DataFrame
df_nameCount.show(truncate = False)
# Create a temporary view of restaurant name counts
df_nameCount.createOrReplaceTempView("Name_Count")

+-----------------------+----------+
name_formatted |name_count|
+-----------------------+----------+
MCDONALDS |870 |
SUBWAYRESTAURANTS |613 |
BURGERKING |337 |
TACOBELL |335 |
PIZZAHUT |330 |
WENDYS |325 |
TIMHORTONS |263 |
SUBWAY |244 |
DOMINOSPIZZA |238 |
KFC |214 |
STARBUCKS |208 |
CHIPOTLEMEXICANGRILL |191 |
JACKINTHEBOX |188 |
PANERABREAD |165 |
JIMMYJOHNS |165 |
POPEYESLOUISIANAKITCHEN|161 |
PANDAEXPRESS |157 |
PAPAJOHNSPIZZA |156 |
ARBYS |148 |
LITTLECAESARSPIZZA |133 |
+-----------------------+----------+
only showing top 20 rows

Next, we determine which of those restaurant names are not among those in our lists from Wikipedia. The result represents potential chains that we should consider for properly answering our client's questions.

In [0]:
# Identify multi-outlet restaurants not included in the Chain_Name list 
df_potentialChain = spark.sql("""
SELECT DISTINCT name_formatted AS potential_chain
           FROM Name_Count AS A
 LEFT ANTI JOIN Chain_Name AS B
             ON A.name_formatted == B.chain
       ORDER BY potential_chain
""")
# Display a count of the records
print("Record count:", df_potentialChain.count())
# Display the resulting DataFrame
df_potentialChain.show(truncate = False)
# Create a temporary view of restaurant name counts
df_potentialChain.createOrReplaceTempView("Potential_Chain")

Record count: 1761
+-----------------------------+
potential_chain |
+-----------------------------+
1000DEGREESNEAPOLITANPIZZERIA|
131MAINRESTAURANT |
1BROTHERSPIZZA |
1HAWAIIANBARBECUE |
3AMIGOS |
3MARGARITAS |
3TOMATOESAMOZZARELLA |
5DINER |
786HALALRESTAURANT |
7ELEVEN |
ABOVETHECRUST |
ABUELOSMEXICANRESTAURANT |
ACMEBURGERCOMPANY |
ADRIANSPIZZA |
AFFYSPREMIUMGRILL |
AFGHANCUISINE |
AGAVE |
AHIPOKI |
AHSOSUSHISTEAK |
AISUSHI |
+-----------------------------+
only showing top 20 rows

We append the chain names list with those discovered within the data. This now comprehensive list of chains will be used for futher forming output DataFrames. Here, we too calculate a field that denotes the restaurant is (N)ot a chain, or is part of an affilliation defined by the (E)xternal chain list, or is (D)erived as a chain from profiling the Yelp data. This will be used as an enhancement to the `df_restaurantData` DataFrame in the next step.

In [0]:
# Append chain restaurants not included in the chain list
df_chainNameCombined = spark.sql("""
        SELECT chain,
               'E' AS chain_source
          FROM Chain_Name
         UNION 
        SELECT potential_chain AS chain,
               'D' AS chain_source
          FROM (SELECT potential_chain
                  FROM Potential_Chain AS A
LEFT ANTI JOIN (SELECT chain
                  FROM Chain_Name) AS B
            ON A.potential_chain == B.chain) AS C
""").orderBy('chain')
# Display a count of the records
print("Record count:", df_chainNameCombined.count())
# Display the resulting DataFrame
df_chainNameCombined.show(truncate = False)
# Replace the temporary view of chain names
df_chainNameCombined.createOrReplaceTempView("Chain_Name")

Record count: 2225
+-----------------------------+------------+
chain |chain_source|
+-----------------------------+------------+
1000DEGREESNEAPOLITANPIZZERIA|D |
131MAINRESTAURANT |D |
1BROTHERSPIZZA |D |
1HAWAIIANBARBECUE |D |
241PIZZA |E |
3AMIGOS |D |
3MARGARITAS |D |
3TOMATOESAMOZZARELLA |D |
5DINER |D |
786HALALRESTAURANT |D |
7ELEVEN |D |
ABOVETHECRUST |D |
ABUELOSMEXICANRESTAURANT |D |
ACMEBURGERCOMPANY |D |
ADRIANSPIZZA |D |
AFFYSPREMIUMGRILL |D |
AFGHANCUISINE |D |
AGAVE |D |
AHIPOKI |D |
AHSOSUSHISTEAK |D |
+-----------------------------+------------+
only showing top 20 rows

###Identifying Chain and Non-chain Restaurants

In the following cell, we enhance the `df_restaurantData` DataFrame with an identifier to denote which are and are not affiliated with a chain by our above deifinition (`is_chain`) and further enable that column to express source of chains' determination.

In [0]:
# List individual chain restaurants and their geocode information
df_restaurantData = spark.sql("""
    SELECT business_id,
           name_formatted,
           name,
           city,
           state,
           postal_code,
           latitude,
           longitude,
           is_chain
      FROM
   (SELECT business_id,
           name_formatted,
           name,
           city,
           state,
           postal_code,
           latitude,
           longitude,
           B.chain_source AS is_chain
      FROM Restaurant_Data AS A
INNER JOIN Chain_Name AS B
        ON A.name_formatted == B.chain
     UNION 
    SELECT business_id,
           name_formatted,
           name,
           city,
           state,
           postal_code,
           latitude,
           longitude,
           'N' AS is_chain
      FROM Restaurant_Data AS C
 ANTI JOIN Chain_Name AS D
        ON C.name_formatted == D.chain) AS E
  ORDER BY name
""")
# Display a count of the records
print("Record count:", df_restaurantData.count())
# Display the resulting DataFrame
df_restaurantData.show(truncate = 75, vertical = True)
# Replace the temporary view of restaurant data
df_restaurantData.createOrReplaceTempView("Restaurant_Data")  

Record count: 63944
-RECORD 0----------------------------------
 business_id | 42yLya7lhe7TQGz3KEdKAw 
 name_formatted | 1HAWAIIANBARBECUE 
 name | #1 Hawaiian Barbecue 
 city | Las Vegas 
 state | NV 
 postal_code | 89119 
 latitude | 36.0819201 
 longitude | -115.1195258 
 is_chain | D 
-RECORD 1----------------------------------
 business_id | gA78_OGp1ekMvZWjRoBmFw 
 name_formatted | 1HAWAIIANBARBECUE 
 name | #1 Hawaiian Barbecue 
 city | Las Vegas 
 state | NV 
 postal_code | 89139 
 latitude | 36.0433730594 
 longitude | -115.2418610396 
 is_chain | D 
-RECORD 2----------------------------------
 business_id | AhD9x2NNFqPvVwmyEZU5Cg 
 name_formatted | 1PHO 
 name | #1 Pho 
 city | Middleburg Heights 
 state | OH 
 postal_code | 44130 
 latitude | 41.3517269 
 longitude | -81.785527 
 is_chain | N 
-RECORD 3----------------------------------
 business_id | kkEqZmVvVkgmCaOqE13mDg 
 name_formatted | 1SUSHI 
 name | #1 Sushi 
 city | Phoenix 
 state | AZ 
 postal_code | 85051 
 latitude | 33.5751273 
 longitude | -112.1208796 
 is_chain | N 
-RECORD 4----------------------------------
 business_id | 4z-QW_f3RwCAxHB5fd58TA 
 name_formatted | 1BROTHERSPIZZA 
 name | #1Brothers Pizza 
 city | Surprise 
 state | AZ 
 postal_code | 85388 
 latitude | 33.6236601054 
 longitude | -112.424706586 
 is_chain | D 
-RECORD 5----------------------------------
 business_id | ncBPf_4toivivsi8b__cRw 
 name_formatted | COMPANYRESTOBAR 
 name | &Company Resto Bar 
 city | Mississauga 
 state | ON 
 postal_code | L5B 3J4 
 latitude | 43.5907682 
 longitude | -79.6363064 
 is_chain | N 
-RECORD 6----------------------------------
 business_id | T6uVlXYp9XeW8U9QajqQdg 
 name_formatted | ONOPOKBAR 
 name | 'ONO Poké Bar 
 city | Toronto 
 state | ON 
 postal_code | M6K 3S2 
 latitude | 43.6395258 
 longitude | -79.4158554 
 is_chain | N 
-RECORD 7----------------------------------
 business_id | 4OfhecFm4wIKoAnciRhtHw 
 name_formatted | TUREL 
 name | 'Turel 
 city | Montréal 
 state | QC 
 postal_code | H2T 1P3 
 latitude | 45.522114 
 longitude | -73.5843931 
 is_chain | N 
-RECORD 8----------------------------------
 business_id | B8GKsrIRRLChyFioejPINg 
 name_formatted | 99CENTPIZZAPLACE 
 name | .99 Cent Pizza Place 
 city | Mesa 
 state | AZ 
 postal_code | 85202 
 latitude | 33.3783921 
 longitude | -111.8722465 
 is_chain | N 
-RECORD 9----------------------------------
 business_id | OtAVXleSatkQeZn9Twg4Aw 
 name_formatted | 00GELATO 
 name | 00 Gelato 
 city | Toronto 
 state | ON 
 postal_code | M5V 1J9 
 latitude | 43.6463119315 
 longitude | -79.391329296 
 is_chain | N 
-RECORD 10---------------------------------
 business_id | FnQ-_pGdw11za0LQ_aORxQ 
 name_formatted | 0109DESSERTCHOCOLATE 
 name | 0109 Dessert & Chocolate 
 city | Toronto 
 state | ON 
 postal_code | M1V 0B3 
 latitude | 43.8142848 
 longitude | -79.2945678 
 is_chain | N 
-RECORD 11---------------------------------
 business_id | qRayy6GSsolpMwqX5VBaIg 
 name_formatted | 1BROTHERSPIZZA 
 name | 1 Brother's Pizza 
 city | Peoria 
 state | AZ 
 postal_code | 85383 
 latitude | 33.7089926 
 longitude | -112.2706753 
 is_chain | D 
-RECORD 12---------------------------------
 business_id | gWY94C5EgNClUOwqJXrBGg 
 name_formatted | 1BROTHERSPIZZA 
 name | 1 Brothers Pizza 
 city | Avondale 
 state | AZ 
 postal_code | 85323 
 latitude | 33.434326 
 longitude | -112.305348 
 is_chain | D 
-RECORD 13---------------------------------
 business_id | 7jgyxiA4e-rikpoOHrPW6g 
 name_formatted | 1HAWAIIANBARBECUE 
 name | 1 Hawaiian Barbecue 
 city | North Las Vegas 
 state | NV 
 postal_code | 89081 
 latitude | 36.266812 
 longitude | -115.116613 
 is_chain | D 
-RECORD 14---------------------------------
 business_id | gS9MphCeLc9JzUrEOEA4_Q 
 name_formatted | 1LOVEKITCHEN 
 name | 1 Love Kitchen 
 city | Toronto 
 state | ON 
 postal_code | M5V 1Z4 
 latitude | 43.6502718 
 longitude | -79.3887054 
 is_chain | N 
-RECORD 15---------------------------------
 business_id | 3

Next, we further enhance `df_restaurantData` to include a column that incicates each restaurant's metro area.

In [0]:
df_restaurantData = spark.sql("""
    SELECT business_id,
           name_formatted,
           name,
           city,
           state,
           postal_code,
           CASE WHEN TRIM(state) == 'QC'
                THEN 'Montreal'
                WHEN TRIM(state) == 'AB'
                THEN 'Calgary'
                WHEN TRIM(state) == 'ON'
                THEN 'Toronto'
                WHEN TRIM(state) == 'PA'
                THEN 'Pittsburgh'
                WHEN TRIM(state) == 'NC'
                THEN 'Charlotte'
                WHEN TRIM(state) == 'SC'
                THEN 'Charlotte'
                WHEN TRIM(state) == 'IL'
                THEN 'Urbana-Champaign'
                WHEN TRIM(state) == 'AZ'
                THEN 'Phoenix'
                WHEN TRIM(state) == 'WI'
                THEN 'Madison'
                WHEN TRIM(state) == 'NV'
                THEN 'Las Vegas'
                WHEN TRIM(state) == 'OH'
                THEN 'Cleveland'
                ELSE 'Undefined'
            END AS metro_area,
           latitude,
           longitude,
           is_chain
      FROM Restaurant_Data
  ORDER BY name
""")
# Display a count of the records
print("Record count:", df_restaurantData.count())
# Display the resulting DataFrame
df_restaurantData.show(truncate = 75, vertical = True)
# Replace the temporary view of restaurant data
df_restaurantData.createOrReplaceTempView("Restaurant_Data")  

Record count: 63944
-RECORD 0----------------------------------
 business_id | 42yLya7lhe7TQGz3KEdKAw 
 name_formatted | 1HAWAIIANBARBECUE 
 name | #1 Hawaiian Barbecue 
 city | Las Vegas 
 state | NV 
 postal_code | 89119 
 metro_area | Las Vegas 
 latitude | 36.0819201 
 longitude | -115.1195258 
 is_chain | D 
-RECORD 1----------------------------------
 business_id | gA78_OGp1ekMvZWjRoBmFw 
 name_formatted | 1HAWAIIANBARBECUE 
 name | #1 Hawaiian Barbecue 
 city | Las Vegas 
 state | NV 
 postal_code | 89139 
 metro_area | Las Vegas 
 latitude | 36.0433730594 
 longitude | -115.2418610396 
 is_chain | D 
-RECORD 2----------------------------------
 business_id | AhD9x2NNFqPvVwmyEZU5Cg 
 name_formatted | 1PHO 
 name | #1 Pho 
 city | Middleburg Heights 
 state | OH 
 postal_code | 44130 
 metro_area | Cleveland 
 latitude | 41.3517269 
 longitude | -81.785527 
 is_chain | N 
-RECORD 3----------------------------------
 business_id | kkEqZmVvVkgmCaOqE13mDg 
 name_formatted | 1SUSHI 
 name | #1 Sushi 
 city | Phoenix 
 state | AZ 
 postal_code | 85051 
 metro_area | Phoenix 
 latitude | 33.5751273 
 longitude | -112.1208796 
 is_chain | N 
-RECORD 4----------------------------------
 business_id | 4z-QW_f3RwCAxHB5fd58TA 
 name_formatted | 1BROTHERSPIZZA 
 name | #1Brothers Pizza 
 city | Surprise 
 state | AZ 
 postal_code | 85388 
 metro_area | Phoenix 
 latitude | 33.6236601054 
 longitude | -112.424706586 
 is_chain | D 
-RECORD 5----------------------------------
 business_id | ncBPf_4toivivsi8b__cRw 
 name_formatted | COMPANYRESTOBAR 
 name | &Company Resto Bar 
 city | Mississauga 
 state | ON 
 postal_code | L5B 3J4 
 metro_area | Toronto 
 latitude | 43.5907682 
 longitude | -79.6363064 
 is_chain | N 
-RECORD 6----------------------------------
 business_id | T6uVlXYp9XeW8U9QajqQdg 
 name_formatted | ONOPOKBAR 
 name | 'ONO Poké Bar 
 city | Toronto 
 state | ON 
 postal_code | M6K 3S2 
 metro_area | Toronto 
 latitude | 43.6395258 
 longitude | -79.4158554 
 is_chain | N 
-RECORD 7----------------------------------
 business_id | 4OfhecFm4wIKoAnciRhtHw 
 name_formatted | TUREL 
 name | 'Turel 
 city | Montréal 
 state | QC 
 postal_code | H2T 1P3 
 metro_area | Montreal 
 latitude | 45.522114 
 longitude | -73.5843931 
 is_chain | N 
-RECORD 8----------------------------------
 business_id | B8GKsrIRRLChyFioejPINg 
 name_formatted | 99CENTPIZZAPLACE 
 name | .99 Cent Pizza Place 
 city | Mesa 
 state | AZ 
 postal_code | 85202 
 metro_area | Phoenix 
 latitude | 33.3783921 
 longitude | -111.8722465 
 is_chain | N 
-RECORD 9----------------------------------
 business_id | OtAVXleSatkQeZn9Twg4Aw 
 name_formatted | 00GELATO 
 name | 00 Gelato 
 city | Toronto 
 state | ON 
 postal_code | M5V 1J9 
 metro_area | Toronto 
 latitude | 43.6463119315 
 longitude | -79.391329296 
 is_chain | N 
-RECORD 10---------------------------------
 business_id | FnQ-_pGdw11za0LQ_aORxQ 
 name_formatted | 0109DESSERTCHOCOLATE 
 name | 0109 Dessert & Chocolate 
 city | Toronto 
 state | ON 
 postal_code | M1V 0B3 
 metro_area | Toronto 
 latitude | 43.8142848 
 longitude | -79.2945678 
 is_chain | N 
-RECORD 11---------------------------------
 business_id | qRayy6GSsolpMwqX5VBaIg 
 name_formatted | 1BROTHERSPIZZA 
 name | 1 Brother's Pizza 
 city | Peoria 
 state | AZ 
 postal_code | 85383 
 metro_area | Phoenix 
 latitude | 33.7089926 
 longitude | -112.2706753 
 is_chain | D 
-RECORD 12---------------------------------
 business_id | gWY94C5EgNClUOwqJXrBGg 
 name_formatted | 1BROTHERSPIZZA 
 name | 1 Brothers Pizza 
 city | Avondale 
 state | AZ 
 postal_code | 85323 
 metro_area | Phoenix 
 latitude | 33.434326 
 longitude | -112.305348 
 is_chain | D 
-RECORD 13---------------------------------
 business_id | 7jgyxiA4e-rikpoOHrPW6g 
 name_formatted | 1HAWAIIANBARBECUE 
 name | 1 Hawaiian Barbecue 
 city | North Las Vegas 
 state | NV 
 postal_code | 89081 
 metro_area | Las Vegas 
 latitude | 36.266812 
 longitude | -115.116613 
 i

With the metro areas designated, we next cleanse the DataFrame of those records that should not actually be part of the dataset. We do so by first identifying those `metro_area`s determined as 'Undefined' from `df_restaurantData`.

In [0]:
# Produce a list of restauraunts that are outside those metro areas of interest ("undefined")
df_restaurantDataUndefined= spark.sql("""
    SELECT business_id,
           name_formatted,
           name,
           city,
           state,
           postal_code,
           metro_area,
           latitude,
           longitude,
           is_chain
      FROM Restaurant_Data
     WHERE metro_area = 'Undefined'
  ORDER BY name
""")
# Display a count of the records
print("Record count:", df_restaurantDataUndefined.count())
# Display the resulting DataFrame
df_restaurantDataUndefined.show(100, truncate = 75, vertical = True)

Record count: 35
-RECORD 0----------------------------------------
 business_id | BmZu5ackYYCrgQIaJ3-r9A 
 name_formatted | ANGELOSPIZZAGRILL 
 name | Angelo's Pizza & Grill 
 city | Rouses Point 
 state | NY 
 postal_code | 12979 
 metro_area | Undefined 
 latitude | 44.9832840757 
 longitude | -73.3683259164 
 is_chain | N 
-RECORD 1----------------------------------------
 business_id | dPlwOjYe6gFT-w85Qg6GeA 
 name_formatted | ATLANTICPLACEDENTALOFFICE 
 name | Atlantic Place Dental Office 
 city | Monterey Park 
 state | CA 
 postal_code | 91754 
 metro_area | Undefined 
 latitude | 36.1250534058 
 longitude | -115.1639175415 
 is_chain | N 
-RECORD 2----------------------------------------
 business_id | 0oyzNutZ--rBqP_boeegPg 
 name_formatted | BESTFRIENDSFAMILYDINER 
 name | Best Friends Family Diner 
 city | Rouses Point 
 state | NY 
 postal_code | 12979 
 metro_area | Undefined 
 latitude | 44.999192 
 longitude | -73.366007 
 is_chain | N 
-RECORD 3----------------------------------------
 business_id | 3zEvzrHRcCuaE_-EWfstMg 
 name_formatted | BURGERKING 
 name | Burger King 
 city | Oakland 
 state | CA 
 postal_code | 94605 
 metro_area | Undefined 
 latitude | 33.4870787575 
 longitude | -112.3632881024 
 is_chain | E 
-RECORD 4----------------------------------------
 business_id | Sm1aTKiMrBRUmpbdVg5wAg 
 name_formatted | CASACAPITANO 
 name | Casa Capitano 
 city | Rouses Point 
 state | NY 
 postal_code | 12979 
 metro_area | Undefined 
 latitude | 44.9833547 
 longitude | -73.3634552 
 is_chain | N 
-RECORD 5----------------------------------------
 business_id | Uz77JHl5e6zlGa-71-6_dA 
 name_formatted | CHINABUFFET 
 name | China Buffet 
 city | Champlain 
 state | NY 
 postal_code | 12919 
 metro_area | Undefined 
 latitude | 44.979672 
 longitude | -73.4449037 
 is_chain | D 
-RECORD 6----------------------------------------
 business_id | FByZsT1Sob5Vf1AYJFPxPg 
 name_formatted | DESIMASALA 
 name | Desi Masala 
 city | Leeds 
 state | XWY 
 postal_code | LS27 8EB 
 metro_area | Undefined 
 latitude | 43.6528212 
 longitude | -79.3763454 
 is_chain | N 
-RECORD 7----------------------------------------
 business_id | 6ipr7nnwrzf_o3MGD-8nlA 
 name_formatted | GINOSPIZZA 
 name | Gino's Pizza 
 city | Rouses Point 
 state | NY 
 postal_code | 12979 
 metro_area | Undefined 
 latitude | 44.9958866 
 longitude | -73.3657087 
 is_chain | D 
-RECORD 8----------------------------------------
 business_id | x5sx84Fa1cpnLwvXGtsJEw 
 name_formatted | HOMEBURGER 
 name | Home Burger 
 city | Hartlepool 
 state | HPL 
 postal_code | TS24 7DA 
 metro_area | Undefined 
 latitude | 43.7429401909 
 longitude | -79.220507741 
 is_chain | N 
-RECORD 9----------------------------------------
 business_id | 8ELdE4e05KUeB10ceSQk3A 
 name_formatted | HOMESTEADSTEAKS 
 name | Homestead Steaks 
 city | Omaha 
 state | NE 
 postal_code | 68114 
 metro_area | Undefined 
 latitude | 36.067414457 
 longitude | -115.0410128385 
 is_chain | N 
-RECORD 10---------------------------------------
 business_id | pyOHNzoMBJjZyihZPWYtFA 
 name_formatted | LASALSITATACOSHOP 
 name | La Salsita Taco Shop 
 city | Avondale 
 state | AR 
 postal_code | 85323 
 metro_area | Undefined 
 latitude | 33.435145 
 longitude | -112.3020797968 
 is_chain | N 
-RECORD 11---------------------------------------
 business_id | f5Pg89x-2xek78E9KHMhNg 
 name_formatted | LAKESIDECOFFEE 
 name | Lakeside Coffee 
 city | Rouses Point 
 state | NY 
 postal_code | 12979 
 metro_area | Undefined 
 latitude | 44.9931808 
 longitude | -73.3641541 
 is_chain | N 
-RECORD 12---------------------------------------
 business_id | 6029Vtm4LVYGPLm00htXrQ 
 name_formatted | MAPLEFIELDSCHAMPIRVING 
 name | Maplefields Champ Irving 
 city | Champlain 
 state | NY 
 postal_code | 12919 
 metro_area | Undefined 
 latitude | 44.9803095616 
 longitude | -73.4547526872 
 is_chain | N 
-RECORD 13---------------------------------------
 business_id | LbM7p-cI0dUCkaUzOyFMTw 
 name_

The resulting list of outlier data is quite manageable to visually verify as not belonging within the dataset of the ten metro areas of interest. As such, we exponge those from the `df_restaurantData` DataFrame. Additionally, we take opportunity to enhance the DataFrame with `country` and `population` fields.

In [0]:
df_restaurantData = spark.sql("""
    SELECT business_id,
           name_formatted,
           name,
           city,
           state,
           postal_code,
           A.metro_area,
           country,
           latitude,
           longitude,
           population,
           is_chain
      FROM Restaurant_Data AS A
INNER JOIN Metro_Population AS B
        ON A.metro_area == B.metro_area
     WHERE A.metro_area <> 'Undefined'
  ORDER BY name
""")
# Display a count of the records
print("Record count:", df_restaurantData.count())
# Display the resulting DataFrame
df_restaurantData.show(truncate = 75, vertical = True)
# Replace the temporary view of restaurant data
df_restaurantData.createOrReplaceTempView("Restaurant_Data")  

Record count: 63909
-RECORD 0----------------------------------
 business_id | gA78_OGp1ekMvZWjRoBmFw 
 name_formatted | 1HAWAIIANBARBECUE 
 name | #1 Hawaiian Barbecue 
 city | Las Vegas 
 state | NV 
 postal_code | 89139 
 metro_area | Las Vegas 
 country | USA 
 latitude | 36.0433730594 
 longitude | -115.2418610396 
 population | 2699000 
 is_chain | D 
-RECORD 1----------------------------------
 business_id | 42yLya7lhe7TQGz3KEdKAw 
 name_formatted | 1HAWAIIANBARBECUE 
 name | #1 Hawaiian Barbecue 
 city | Las Vegas 
 state | NV 
 postal_code | 89119 
 metro_area | Las Vegas 
 country | USA 
 latitude | 36.0819201 
 longitude | -115.1195258 
 population | 2699000 
 is_chain | D 
-RECORD 2----------------------------------
 business_id | AhD9x2NNFqPvVwmyEZU5Cg 
 name_formatted | 1PHO 
 name | #1 Pho 
 city | Middleburg Heights 
 state | OH 
 postal_code | 44130 
 metro_area | Cleveland 
 country | USA 
 latitude | 41.3517269 
 longitude | -81.785527 
 population | 1763000 
 is_chain | N 
-RECORD 3----------------------------------
 business_id | kkEqZmVvVkgmCaOqE13mDg 
 name_formatted | 1SUSHI 
 name | #1 Sushi 
 city | Phoenix 
 state | AZ 
 postal_code | 85051 
 metro_area | Phoenix 
 country | USA 
 latitude | 33.5751273 
 longitude | -112.1208796 
 population | 4511000 
 is_chain | N 
-RECORD 4----------------------------------
 business_id | 4z-QW_f3RwCAxHB5fd58TA 
 name_formatted | 1BROTHERSPIZZA 
 name | #1Brothers Pizza 
 city | Surprise 
 state | AZ 
 postal_code | 85388 
 metro_area | Phoenix 
 country | USA 
 latitude | 33.6236601054 
 longitude | -112.424706586 
 population | 4511000 
 is_chain | D 
-RECORD 5----------------------------------
 business_id | ncBPf_4toivivsi8b__cRw 
 name_formatted | COMPANYRESTOBAR 
 name | &Company Resto Bar 
 city | Mississauga 
 state | ON 
 postal_code | L5B 3J4 
 metro_area | Toronto 
 country | CAN 
 latitude | 43.5907682 
 longitude | -79.6363064 
 population | 6472000 
 is_chain | N 
-RECORD 6----------------------------------
 business_id | T6uVlXYp9XeW8U9QajqQdg 
 name_formatted | ONOPOKBAR 
 name | 'ONO Poké Bar 
 city | Toronto 
 state | ON 
 postal_code | M6K 3S2 
 metro_area | Toronto 
 country | CAN 
 latitude | 43.6395258 
 longitude | -79.4158554 
 population | 6472000 
 is_chain | N 
-RECORD 7----------------------------------
 business_id | 4OfhecFm4wIKoAnciRhtHw 
 name_formatted | TUREL 
 name | 'Turel 
 city | Montréal 
 state | QC 
 postal_code | H2T 1P3 
 metro_area | Montreal 
 country | CAN 
 latitude | 45.522114 
 longitude | -73.5843931 
 population | 4221000 
 is_chain | N 
-RECORD 8----------------------------------
 business_id | B8GKsrIRRLChyFioejPINg 
 name_formatted | 99CENTPIZZAPLACE 
 name | .99 Cent Pizza Place 
 city | Mesa 
 state | AZ 
 postal_code | 85202 
 metro_area | Phoenix 
 country | USA 
 latitude | 33.3783921 
 longitude | -111.8722465 
 population | 4511000 
 is_chain | N 
-RECORD 9----------------------------------
 business_id | OtAVXleSatkQeZn9Twg4Aw 
 name_formatted | 00GELATO 
 name | 00 Gelato 
 city | Toronto 
 state | ON 
 postal_code | M5V 1J9 
 metro_area | Toronto 
 country | CAN 
 latitude | 43.6463119315 
 longitude | -79.391329296 
 population | 6472000 
 is_chain | N 
-RECORD 10---------------------------------
 business_id | FnQ-_pGdw11za0LQ_aORxQ 
 name_formatted | 0109DESSERTCHOCOLATE 
 name | 0109 Dessert & Chocolate 
 city | Toronto 
 state | ON 
 postal_code | M1V 0B3 
 metro_area | Toronto 
 country | CAN 
 latitude | 43.8142848 
 longitude | -79.2945678 
 population | 6472000 
 is_chain | N 
-RECORD 11---------------------------------
 business_id | qRayy6GSsolpMwqX5VBaIg 
 name_formatted | 1BROTHERSPIZZA 
 name | 1 Brother's Pizza 
 city | Peoria 
 state | AZ 
 postal_code | 85383 
 metro_area | Phoenix 
 country | USA 
 latitude | 33.7089926 
 longitude | -112.2706753 
 population | 4511000 
 is_chain | D 
-RECORD 12---------------------------------
 business_id | gWY94C5EgNClUOwqJXrBGg 
 name_formatted |

A flag field to denote each restaurant as (N)on-chain, (L)ocal chain, or (M)ulti-metro area chain affiliation is now added to `df_restaurantData`. This will provide opportunity for higher evaluation of trends or patterns found in our primary output.

In [0]:
# Determine which chains span multiple metro areas (Query B). Then, use that result to calculate a new field to denote multi-metro chains. (Query A). Finally, from that, calculate a field to indicate the affiliation of the restaurant.
df_restaurantData = spark.sql("""
SELECT business_id,
       name_formatted,
       name,
       city,
       state,
       postal_code,
       metro_area,
       country,
       latitude,
       longitude,
       population,
       is_chain,
       CASE WHEN is_chain = 'N'
            THEN 'N'
            ELSE CASE WHEN is_multi_metro = 'Y'
                      THEN 'M'
                      ELSE 'L'
                      END
       END AS affiliation
    FROM (SELECT business_id,
                 A.name_formatted,
                 name,
                 city,
                 state,
                 postal_code,
                 metro_area,
                 country,
                 latitude,
                 longitude,
                 population,
                 is_chain,
                 CASE WHEN B.name_formatted IS NULL
                      THEN 'N'
                      ELSE 'Y'
                  END AS is_multi_metro
            FROM Restaurant_Data AS A
 LEFT OUTER JOIN (SELECT name_formatted
                    FROM (SELECT name_formatted,
                                 COUNT(metro_area) AS metro_area_count
                            FROM (SELECT DISTINCT name_formatted,
                                                  metro_area
                                             FROM Restaurant_Data
                                            WHERE is_chain <> 'N') AS A
                                         GROUP BY name_formatted
                                           HAVING metro_area_count > 1)) AS B
               ON A.name_formatted = B.name_formatted) AS C
""")
# Display a count of the records
print("Record count:", df_restaurantData.count())
# Display the resulting DataFrame
df_restaurantData.show(truncate = 75, vertical = True)
# Replace the temporary view of restaurant data
df_restaurantData.createOrReplaceTempView("Restaurant_Data")

Record count: 63909
-RECORD 0----------------------------------------
 business_id | hIY1X7BjFwn0Xp5srv_VCA 
 name_formatted | RESTAURANTDICANNS 
 name | Restaurant Dic Ann's 
 city | Saint-Jérôme 
 state | QC 
 postal_code | J7Y 2G4 
 metro_area | Montreal 
 country | CAN 
 latitude | 45.7793419 
 longitude | -74.0150144 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 1----------------------------------------
 business_id | ctwh8bLtYm821qz6mhcDVQ 
 name_formatted | TRATTORIAGIO 
 name | Trattoria Gio 
 city | Montréal 
 state | QC 
 postal_code | H2Y 
 metro_area | Montreal 
 country | CAN 
 latitude | 45.5079 
 longitude | -73.5538 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 2----------------------------------------
 business_id | 09enlT7b86opzmgL-8Tz5A 
 name_formatted | RESTAURANTSDLAFLEUR 
 name | Restaurants D Lafleur 
 city | Montréal 
 state | QC 
 postal_code | H2X 3L7 
 metro_area | Montreal 
 country | CAN 
 latitude | 45.517817 
 longitude | -73.5699549 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 3----------------------------------------
 business_id | -sSUtare_A_jWarLr3B20A 
 name_formatted | RESTAURANTNICPIC 
 name | Restaurant Nic-Pic 
 city | Saint-Jean-Sur-Richelieu 
 state | QC 
 postal_code | J3B 5H9 
 metro_area | Montreal 
 country | CAN 
 latitude | 45.3040228529 
 longitude | -73.2667620275 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 4----------------------------------------
 business_id | 8eZnFk390ISLsKBGhSr-sA 
 name_formatted | LACOLONIE 
 name | La Colonie 
 city | Boucherville 
 state | QC 
 postal_code | J4B 1A9 
 metro_area | Montreal 
 country | CAN 
 latitude | 45.579576 
 longitude | -73.447931 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 5----------------------------------------
 business_id | WqwEbfAYIXJjUXEAlo1QYg 
 name_formatted | RESTPOUTINE 
 name | Rest-Poutine 
 city | Blainville 
 state | QC 
 postal_code | J7C 2L5 
 metro_area | Montreal 
 country | CAN 
 latitude | 45.6571466 
 longitude | -73.8540778 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 6----------------------------------------
 business_id | Ek4sVcvmACWp8P1yizjwZg 
 name_formatted | QUEUEDECHEVAL 
 name | Queue de Cheval 
 city | Montréal 
 state | QC 
 postal_code | H3G 1Z2 
 metro_area | Montreal 
 country | CAN 
 latitude | 45.4972995 
 longitude | -73.5731032 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 7----------------------------------------
 business_id | kkXmJbAzH3PlxSVRWcNiyg 
 name_formatted | TANDOORIBELLEVUE 
 name | Tandoori Bellevue 
 city | Sainte-Anne-de-Bellevue 
 state | QC 
 postal_code | H9X 1L5 
 metro_area | Montreal 
 country | CAN 
 latitude | 45.4033354 
 longitude | -73.9501345 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 8----------------------------------------
 business_id | n4uDGwRGtJ3vW2Adbha_wg 
 name_formatted | RESTAURANTDECCA77 
 name | Restaurant Decca 77 
 city | Montréal 
 state | QC 
 postal_code | H3B 4X4 
 metro_area | Montreal 
 country | CAN 
 latitude | 45.4973248 
 longitude | -73.5711434 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 9----------------------------------------
 business_id | MIwZFWqm8jytQ0Or01EWkQ 
 name_formatted | TANDOORIMAHAL 
 name | Tandoori Mahal 
 city | Montréal 
 state | QC 
 postal_code | H2S 1J1 
 metro_area | Montreal 
 country | CAN 
 latitude | 45.5426035 
 longitude | -73.6071175 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 10---------------------------------------
 business_id | dkERhCfr5q0Qcbh7a6Tecw 
 name_formatted | BREAKFASTCLUBMOULERIELE 
 name | Breakfast Club & Moulerie (Le) 
 city | Brossard 
 state | QC 
 postal_code | J4Y 1A2 
 metro_area | Montreal 
 country | CAN 
 latitude | 45.4570934 
 longitude | -73.4641697 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 11---------------------------------------
 business_id | a1nHsV

#Part 3: Outputting the Data

With our data set optimized, we now export its extracts for fulfilling the vision - yielding information in a comprehensive way to the client, and answering the questions asked at the begining of this notebook.

##Part 3A: Generate Tables for Export to Tableau

In the following cells, we generate sets of data which we export to Tableau. There, the data are used to produce meaningful visualization and re-imported to this notebook for dispaly as images. We first generate a table of data related to the restaurants exclusively.

In [0]:
# Assemle a DataFrame of statistics regarding restaurants for output as a table.
df_restaurantDataOutput = spark.sql("""
  SELECT business_id,
         name,
         metro_area,
         country,
         population,
         is_chain,
         affiliation
    FROM Restaurant_Data
""")
# Display a count of the records
print("Record count:", df_restaurantData.count())
# Display the resulting DataFrame
df_restaurantDataOutput.show(truncate = 75, vertical = True)
# Write the DataFrame to a table object
df_restaurantDataOutput.write.mode("overwrite").saveAsTable("Restaurant_Data_Output")

Record count: 63909
-RECORD 0-------------------------------------
 business_id | hIY1X7BjFwn0Xp5srv_VCA 
 name | Restaurant Dic Ann's 
 metro_area | Montreal 
 country | CAN 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 1-------------------------------------
 business_id | ctwh8bLtYm821qz6mhcDVQ 
 name | Trattoria Gio 
 metro_area | Montreal 
 country | CAN 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 2-------------------------------------
 business_id | 09enlT7b86opzmgL-8Tz5A 
 name | Restaurants D Lafleur 
 metro_area | Montreal 
 country | CAN 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 3-------------------------------------
 business_id | -sSUtare_A_jWarLr3B20A 
 name | Restaurant Nic-Pic 
 metro_area | Montreal 
 country | CAN 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 4-------------------------------------
 business_id | 8eZnFk390ISLsKBGhSr-sA 
 name | La Colonie 
 metro_area | Montreal 
 country | CAN 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 5-------------------------------------
 business_id | WqwEbfAYIXJjUXEAlo1QYg 
 name | Rest-Poutine 
 metro_area | Montreal 
 country | CAN 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 6-------------------------------------
 business_id | Ek4sVcvmACWp8P1yizjwZg 
 name | Queue de Cheval 
 metro_area | Montreal 
 country | CAN 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 7-------------------------------------
 business_id | kkXmJbAzH3PlxSVRWcNiyg 
 name | Tandoori Bellevue 
 metro_area | Montreal 
 country | CAN 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 8-------------------------------------
 business_id | n4uDGwRGtJ3vW2Adbha_wg 
 name | Restaurant Decca 77 
 metro_area | Montreal 
 country | CAN 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 9-------------------------------------
 business_id | MIwZFWqm8jytQ0Or01EWkQ 
 name | Tandoori Mahal 
 metro_area | Montreal 
 country | CAN 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 10------------------------------------
 business_id | dkERhCfr5q0Qcbh7a6Tecw 
 name | Breakfast Club & Moulerie (Le) 
 metro_area | Montreal 
 country | CAN 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 11------------------------------------
 business_id | a1nHsVk-0dPherQldZCSSQ 
 name | Capitaine Fish 
 metro_area | Montreal 
 country | CAN 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 12------------------------------------
 business_id | p94oHG7K9BqcMVVOlkzhPg 
 name | larrys 
 metro_area | Montreal 
 country | CAN 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 13------------------------------------
 business_id | iU-Xhq8BXiWHirjYqNkxgg 
 name | Panzzerotti 
 metro_area | Montreal 
 country | CAN 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 14------------------------------------
 business_id | -1xuC540Nycht_iWFeJ-dw 
 name | Romados 
 metro_area | Montreal 
 country | CAN 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 15------------------------------------
 business_id | SCqI6lRq-coK8vZlB643kw 
 name | Restaurant Onyx 
 metro_area | Montreal 
 country | CAN 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 16------------------------------------
 business_id | ovZdUDpft8vKgsSpCX7kIg 
 name | Imperio 
 metro_area | Montreal 
 country | CAN 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 17------------------------------------
 business_id | 7HnlXTF9f29HOYfblGLIHQ 
 name | Toyo 
 metro_area | Montreal 
 country | CAN 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 18------------------------------------
 business_id | 3FfEjcnsERHlDRlJ_w-xbw 
 name | Med Pizza 
 metro_area | Montreal 
 country | CAN 
 population | 4221000 
 is_chain | N 
 affiliation | N 
-RECORD 19------------------------------------
 business_id | RRKF775V

Next, we generate a table of records that join the restaurant data with their review data.

In [0]:
# Combine the df_restaurantData and df_reveiwData DataFrames for output as a table.
df_outputData = spark.sql("""
    SELECT A.business_id,
           name,
           metro_area,
           country,
           population,
           is_chain,
           affiliation,
           user_id,
           SUM(B.cool) AS cool_vote, 
           SUM(B.funny) AS funny_vote, 
           SUM(B.useful) AS useful_vote, 
           SUM(B.cool) + SUM(B.funny) + SUM(B.useful) AS total_vote,
           subtotal_review
      FROM (SELECT business_id,
                   name,
                   name_formatted,
                   city,
                   state,
                   postal_code,
                   metro_area,
                   country,
                   latitude,
                   longitude,
                   population,
                   is_chain,
                   affiliation
              FROM Restaurant_Data) AS A
INNER JOIN (  SELECT business_id,
                     user_id,
                     SUM(cool) AS cool, 
                     SUM(funny) AS funny, 
                     SUM(useful) AS useful,
                     subtotal_review
                FROM Review_Data
            GROUP BY business_id, user_id, subtotal_review) AS B
        ON A.business_id == B.business_id
  GROUP BY A.business_id,
           name,
           name_formatted,
           city,
           state,
           postal_code,
           metro_area,
           country,
           latitude,
           longitude,
           population,
           is_chain,
           affiliation,
           user_id,
           subtotal_review
  ORDER BY business_id, user_id
  """)
# Display a count of the records
print("Record count:", df_outputData.count())
# Display the resulting DataFrame
df_outputData.show(truncate = 75, vertical = True)
# Write the DataFrame to a table object
df_outputData.write.mode("overwrite").saveAsTable("Output_Data")

Record count: 4881612
-RECORD 0---------------------------------
 business_id | --1UhMGODdWsrMastO9DZw 
 name | The Spicy Amigos 
 metro_area | Calgary 
 country | CAN 
 population | 1547000 
 is_chain | N 
 affiliation | N 
 user_id | 1b4rlFFdNfeysOjbrFykng 
 cool_vote | 0 
 funny_vote | 0 
 useful_vote | 0 
 total_vote | 0 
 subtotal_review | 29 
-RECORD 1---------------------------------
 business_id | --1UhMGODdWsrMastO9DZw 
 name | The Spicy Amigos 
 metro_area | Calgary 
 country | CAN 
 population | 1547000 
 is_chain | N 
 affiliation | N 
 user_id | 4YHZzwlxEMi7zWO6Osszdw 
 cool_vote | 0 
 funny_vote | 0 
 useful_vote | 0 
 total_vote | 0 
 subtotal_review | 29 
-RECORD 2---------------------------------
 business_id | --1UhMGODdWsrMastO9DZw 
 name | The Spicy Amigos 
 metro_area | Calgary 
 country | CAN 
 population | 1547000 
 is_chain | N 
 affiliation | N 
 user_id | A4bpHuvzaQt9-XAg8e9Msw 
 cool_vote | 0 
 funny_vote | 0 
 useful_vote | 0 
 total_vote | 0 
 subtotal_review | 29 
-RECORD 3---------------------------------
 business_id | --1UhMGODdWsrMastO9DZw 
 name | The Spicy Amigos 
 metro_area | Calgary 
 country | CAN 
 population | 1547000 
 is_chain | N 
 affiliation | N 
 user_id | BgTWMo2qRrXINPiM35w8zw 
 cool_vote | 1 
 funny_vote | 0 
 useful_vote | 1 
 total_vote | 2 
 subtotal_review | 29 
-RECORD 4---------------------------------
 business_id | --1UhMGODdWsrMastO9DZw 
 name | The Spicy Amigos 
 metro_area | Calgary 
 country | CAN 
 population | 1547000 
 is_chain | N 
 affiliation | N 
 user_id | Bsy9F-59sl9OT_bvZNl3hA 
 cool_vote | 0 
 funny_vote | 0 
 useful_vote | 0 
 total_vote | 0 
 subtotal_review | 29 
-RECORD 5---------------------------------
 business_id | --1UhMGODdWsrMastO9DZw 
 name | The Spicy Amigos 
 metro_area | Calgary 
 country | CAN 
 population | 1547000 
 is_chain | N 
 affiliation | N 
 user_id | FYhU1fKQ7n11WQ7gcFYOag 
 cool_vote | 0 
 funny_vote | 0 
 useful_vote | 0 
 total_vote | 0 
 subtotal_review | 29 
-RECORD 6---------------------------------
 business_id | --1UhMGODdWsrMastO9DZw 
 name | The Spicy Amigos 
 metro_area | Calgary 
 country | CAN 
 population | 1547000 
 is_chain | N 
 affiliation | N 
 user_id | GL81ktDIteXA2VVH6gIakg 
 cool_vote | 2 
 funny_vote | 0 
 useful_vote | 1 
 total_vote | 3 
 subtotal_review | 29 
-RECORD 7---------------------------------
 business_id | --1UhMGODdWsrMastO9DZw 
 name | The Spicy Amigos 
 metro_area | Calgary 
 country | CAN 
 population | 1547000 
 is_chain | N 
 affiliation | N 
 user_id | Le81nQXs_LwAxRk9eK8oQQ 
 cool_vote | 0 
 funny_vote | 0 
 useful_vote | 0 
 total_vote | 0 
 subtotal_review | 29 
-RECORD 8---------------------------------
 business_id | --1UhMGODdWsrMastO9DZw 
 name | The Spicy Amigos 
 metro_area | Calgary 
 country | CAN 
 population | 1547000 
 is_chain | N 
 affiliation | N 
 user_id | NBQnPd9dhwRkCvZRg_SzkQ 
 cool_vote | 0 
 funny_vote | 0 
 useful_vote | 1 
 total_vote | 1 
 subtotal_review | 29 
-RECORD 9---------------------------------
 business_id | --1UhMGODdWsrMastO9DZw 
 name | The Spicy Amigos 
 metro_area | Calgary 
 country | CAN 
 population | 1547000 
 is_chain | N 
 affiliation | N 
 user_id | NoQCmYKyMPs4D01Wa6dZew 
 cool_vote | 0 
 funny_vote | 0 
 useful_vote | 0 
 total_vote | 0 
 subtotal_review | 29 
-RECORD 10--------------------------------
 business_id | --1UhMGODdWsrMastO9DZw 
 name | The Spicy Amigos 
 metro_area | Calgary 
 country | CAN 
 population | 1547000 
 is_chain | N 
 affiliation | N 
 user_id | NqpKiaRsGfuU2voV5dPRCQ 
 cool_vote | 2 
 funny_vote | 0 
 useful_vote | 5 
 total_vote | 7 
 subtotal_review | 29 
-RECORD 11--------------------------------
 business_id | --1UhMGODdWsrMastO9DZw 
 name | The Spicy Amigos 
 metro_area | Calgary 
 country | CAN 
 population | 1547000 
 is_chain | N 
 affiliation | N 
 user_id | P3V5p1Wt3znW2-4YtgafbA 
 cool_vote | 0 
 funny_vote | 0 
 useful_vote | 0 
 total_vote | 0 
 subtotal_review | 29 
-RECORD 12--------------------

##Part 3B: Display Tableau Output within this Notebook
After visuals are produced for the data using Tableau, they are reintroduced below as images.

The two cells that follow:
<ul>
  <li>create a directory to receive and store image files.</li>
  <li>manage the importation of the chart produced in Tableau into that directory.</li>
  <li>convert the chart from its dynamic state into an image file.</li>
</ul>

In [0]:
from PIL import Image
TEMP_DIR = "/temp"
 
def getWidth(path):
  with Image.open(path) as img:
    width, height = img.size
    return(width)
  
def getDbfsPathName(path):
    # Get the fileinfo containing the path and name
  if path.startswith("/dbfs") != True:
    raise Exception("The path provided does not start with /dbfs")
  new_path = "dbfs:" + path[5:]
  # get the file info for the path
  file_list = dbutils.fs.ls(new_path)
  if len(file_list) != 1:
    raise Exception("The path provided is not a single file on dbfs")
  dbfs_path = file_list[0].path
  filename = file_list[0].name
  return(dbfs_path, filename)
  
def getTempPath(filename):
  # Create the temp directory if it does not exist
  temp_path = "file:" + TEMP_DIR
  dbutils.fs.mkdirs(temp_path)
  temp_list = dbutils.fs.ls(temp_path)
  # get a name to use for the copy
  temp_files = []
  for info in temp_list:
    temp_files.append(info.name)
  increment = 0
  new_name = filename
  while new_name in temp_files:
    increment+=1
    new_name = filename + "." + str(increment)
  access_path = TEMP_DIR + "/" + new_name # used for file opening
  return(access_path)

In [0]:
import base64
from PIL import Image
 
def showimage(path, width=0):
  image_string = ""
  img_tag = ""
  dbfs_path, filename = getDbfsPathName(path)
  access_path = getTempPath(filename)
  # copy the file
  copy_path = "file:" + access_path
  dbutils.fs.cp(dbfs_path,copy_path)
  with open(access_path, "rb") as image_file:
    image_string = base64.b64encode(image_file.read() ).decode('utf-8') 
    
  # Is the width setting a positive integer?  A width of 50 means 50%
  if width > 0 and width < 1:
    print("If the width parameter is specified, it must be 1 or more.  A width of 50 means 50%. The width entered was " + str(width) + ", so the original image width was used.")
    width = 0 #reset
    
  if width == 0:
    height = 0
    # Get the width and height of the image in pixels
    with Image.open(access_path) as img:
      width, height = img.size
      
    framewidth = width * 1.1
    # Build the image tag
    img_tag = '''
    <style>
    div {
      min-width: %ipx;
      max-width: %ipx;
    }
    </style>
    <div><img src="data:image/png;base64, %s"  style="width:%ipx;height=%ipx;" /></div>''' % (framewidth,framewidth,image_string, width, height)
  else: # a width was specified
    originalWidth = getWidth(access_path)
    imagewidth = int( width / 100.0 * originalWidth)
    framewidth = int( imagewidth * 1.1 )
    # Build the image tag
    img_tag = '''
    <style>
    div {
      min-width: %ipx;
      max-width: %ipx;
    }
    </style>
    <div><img src="data:image/png;base64, %s"  width="%ipx" height="auto"></div>''' % (framewidth,framewidth,image_string, imagewidth)
  # Clean up the file
  dbutils.fs.rm(copy_path)
  return(img_tag)


###Answer 1: Do Reviews of Non-chain Restaurants Get More Votes or More Often Get Voted on than Reviews of Chain Restaurants?

To produce an insightful response, we present the data in the form of their distribution by chain status and metro area.

From creation of the chart below, we indeed see a pattern: across all metro areas, non-chain restaurant reviews receive a mean of about half a vote more than do chain restaurant reviews. <i>In a graph further down the notebook we demonstrat that the total votes cast does not create an impact to this distribution among markets.</i> 

We also observe that votes for non-chain restaurant reviews distribute, on average, a full vote beyond those of non-chain restaurant reviews in the same metro area. While this is a useful answer to the business question, it inspires us to look deeper into the components of the Chain columns. First, what propotion of those chain restaurants are identified as the banners we obtained from Wikipedia and what proportion are novel - derived from the rule we have applied to the Yelp data itself? Second, What percentage of each metro area's restaurants are chains which are unique to that geography?

<i><b>How we created this visual:</b> In the Tableau interface, we create a calculated dimension to consolidate the three possible values of `Is Chain` into two - merely parsing chain from non-chain restaurants. We then add it, secondary, with the `Metro Area` dimension into the Columns shelf. Next, we create a calculated measure dividing `Total Vote` by `Subtotal Review` (as `Weighted Vote`) and add it as to the Tableau Rows shelf. Now, we add the `Business Id` dimension into the Details card and set the Shapes to circles. Finally, we edit the `Weighted Vote` pill in the Rows shelf into an `If` statment that caps the maximum Votes Per Review at six. This presents the box and whisker portion of the distribution to maximum focus.</i>

In [0]:
displayHTML( showimage("/dbfs/FileStore/tables/DistributionOfVotesPerReviewByMetroArea.png", 78) )

With the chart below, we verify that the number of chain restauraunts comprise the minority of total restaurants in each metro area. Of particular interest here is Montreal. As one of the highest populated metro areas, it features the lowest proportion of chain restaurants (under 20%). Further, in all Canada metro areas, chains as determined by the lists we use from Wikipedia are outweighed significantly by chains we derived from the Yelp data (three or more same-named outlets).

<i><b>How we created this visual:</b> In the Tableau interface, we add the `Metro Area` dimension into the Columns shelf and the `Business Id Count` measure into the Rows shelf. We next drag the `Is Chain` dimension into the Color card. Then, we set the "Quick Table Calculation" setting for `Business Id Count` to "Percent of Total." This establishes the output to a 100% stacked bar chart. Next, we add to the Rows shelf the `Population` measure. This creates a second chart that we set as a line type. Finally, we set the `Population` pill's drop option for "Dual Axis." This overlays the line chart atop the bar data.</i>

In [0]:
displayHTML( showimage("/dbfs/FileStore/tables/RestaurantsByMetroArea_Determinant.png", 75) )

Creating the following chart, we can view the makeup of chain restaurants by their transcendence across metro areas. Notable here is the pattern of local chains (those unique to one metro area) increase as a percentage of all restaurants, commensurate with metro area populations. Yet, the percentage of multi-metro chains do not follow such a trend. 

<i><b>How we created this visual:</b> In the Tableau interface, we add the `Metro Area` dimension into the Columns shelf and the `Business Id Count` measure into the Rows shelf. We next add the `Affiliate` dimension into the Color card. Then, we set the "Quick Table Calculation" setting for `Business Id Count` to "Percent of Total." This estalishes the output to a 100% stacked bar chart. Next, we add to the Rows shelf the `Population` measure. This creates a second chart that we set as a line type. Finally, we set the `Population` pill's drop option for "Dual Axis." This overlays the line chart atop the bar data.</i>

In [0]:
displayHTML( showimage("/dbfs/FileStore/tables/RestaurantsByMetroArea_Affiliation.png", 75) )

###Answer 2: Are there different patterns for "funny," "useful," or "cool" votes?

To effetively reveal any pattern, a stacked bar graph is designed to show vote type percentage by chain status and metro area..

The following chart displays the distribution of vote type per metro area, while also showing the difference between chain and non-chain restaurants. To help answer the second question, we see a pattern of "useful" being the most common vote type accross the board, with chains receiving a lower percentage of "useful" votes in comparison to non-chain restaurants. With the Total Vote shown as well, we observe two interesting things. First, the number of votes cast overall has no perceivable inmpact on the distribution of their type. Also, while each metro area demonstrates favor of votes cast to non-chain reviews, both Las Vegas and Phoenix seem to draw an exceptional interest among users in offering votes to non-chain reviews.

<i><b>How we created this visual:</b> In Tableau, we add the `Metro Area` dimension as well as the grouped `Is Chain` dimension into the Columns shelf. Additionally, we add the `Measure Value` measure into the Rows shelf. `Measure Value` contains three calculated fields for "Cool", "Funny", and "Useful" votes, counting their percent makeup overall. We next add the `Measure Name` dimension into the Color card, allowing us to change the format the Y-axis to a percentage. Finally, we add the `Total Vote` measure to the Rows shelf, narrow its bar width, and then check that pill's Dual Axis setting to overlay the two visuals.</i>

In [0]:
displayHTML( showimage("/dbfs/FileStore/tables/VoteTypeDistributionByMetroArea.png", 77) )

###Answer 3: Does this vary geographically? In other words, are there just more users voting in a big city?

Through the construct of the above charts, we indirectly answer this question. By weighting the votes per review in the first visual, we leveled that measure, filtering undue bias of population influence. Likewise, in the fourth visual, the type of vote cast for reviews is proportionate to the number of reviews in aggregate for that metro area. These methods for reflecting the respective data give us assurance that their output offer genuine, usable, information.

#Determination and Caveats

Through this notebook, we have demonstrated the key steps to effectively answer questions a business would ask from a big data set. We did so in three key phases:
<ul>
  <li>Accessing primary data sets internal to the business, and incorporating complimentary data from other sources</li>
  <li>Wrangle and refine those data into a focused and comprehensive tables</li>
  <li>From the tables, produce intuitive visuals that address the questions asked by the business</li>
</ul>

In the exercise, we have learned that Yelp review readers tend to cast more votes to reviews written about non-chain restaurants, and they tend to favor the "useful" vote in that process. Further, we can state confidently that those trends are not influenced by the size of the metro area in which those reviews are posted. From these output, we are inspired to wonder if adjustment to Yelp's voting system would change these patterns in a way to grow the business.

While this process proved effective in those answers, it also revealed some limitations of the data that must be disclosed when presenting results: 
<ul>
  <li>Although great care was taken to focus to genuine restaurant businesses, variance in that grouping could exist.
    <ul>
      <li>A paucity of businesses have no categorization.</li>
      <li>Some of the businesses collected/omitted for the output could be mis-categorized within their Yelp profile.</li>
    </ul>
  <li>Our methodology for determining the definition of a "chain" restaurant can be challenged.</li>
  <ul>
    <li>By augmenting the data to parse derived chain restaurants from listed chains, the above code can be adjusted to alter the determination of chains or filter undesired portions from the output.
  </ul>
  <li>Canada metro-areas are comparatively less data-rich than those in the US due to Yelp entering Canada in 2009 (five years after its founding in America).</li>
  <ul>
    <li>The ten-plus years of Canada data available are robust enough to reflect meaningful metrics, and this will only grow as new Canada data come into the system over the years.
  </ul>
</ul>